# Import

In [169]:
import tkinter as tk
from tkinter import ttk
import sys
from datetime import datetime
# import sys
date = datetime.today().strftime('%Y-%m-%d')

In [170]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait as wait

In [171]:
import pandas as pd
import win32api
import string
import os
import re
import fnmatch
import shutil
from time import sleep
import random
import fitz
from fitz import Document, Page, Rect
import xlwings as xw
from xlwings.constants import DeleteShiftDirection
import threading
import subprocess as sp

In [172]:
# import tkinter as tk
from tkinter import *
# from tkinter import ttk
from tkinter import filedialog
# from tkinter import font as kjktkFont
from tkinter import simpledialog

# Tracking Variable

## def req_form_value_track

In [173]:
def req_form_value_track(sv):
    # print(len(sv.get()))
    if len(sv.get()) > 4:
        btn_write_script["state"] = NORMAL
    else:
        btn_write_script["state"] = DISABLED

## def script_file_path_track

In [174]:
def script_file_path_track(sv):
    # print(len(sv.get()))
    global script_file
    if len(sv.get()) > 4:
        script_file = entry_script_file.get().strip()
        btn_read_script["state"] = NORMAL
        print('var_script_file_str',entry_script_file.get())
        if len(entry_ECO_num.get()) > 4:
            btn_add_items["state"] = NORMAL
            btn_update_BOM["state"] = NORMAL
        # else:
        #     btn_add_items["state"] = DISABLED
    else:
        btn_read_script["state"] = DISABLED
        btn_add_items["state"] = DISABLED
        btn_update_BOM["state"] = DISABLED

## def_ECO_number_track

In [175]:
def def_ECO_number_track(sv):
    # print(len(sv.get()))
    # global ECO_number
    if len(sv.get()) > 4:
        # ECO_number = entry_ECO_num.get().strip()
        # print('ECO_number tracking: ',ECO_number)
        if len(entry_script_file.get()) > 4:
            btn_add_items["state"] = NORMAL
            btn_update_BOM["state"] = NORMAL
        btn_goto_ECO["state"] = NORMAL
        # btn_update_sap["state"] = NORMAL
        btn_down_report["state"] = NORMAL

    elif len(sv.get()) < 5:
        # ECO_number = None
        btn_add_items["state"] = DISABLED
        btn_update_BOM["state"] = DISABLED
        btn_goto_ECO["state"] = DISABLED
        btn_update_sap["state"] = DISABLED
        btn_down_report["state"] = DISABLED

# Script frame

## def get_form_file

In [176]:
### get form file
def get_form_file():
    request_form_file_path = filedialog.askopenfilename(title= "Please select ECO Request Form file",\
                    filetypes=[("Excel files", ".xlsx .xls .csv .xlsm")])
    print('len of ECO request form path: ',len(request_form_file_path))
    if len(request_form_file_path) > 3:
        entry_req_form.delete(0,END)
        entry_req_form.insert(0,request_form_file_path)
        
    # return request_form_file_path

### check box download
def isChecked():
    print(f'check box is {var_check_down.get()}')
#     return Label(frame_script, text=f'Checkbutton is checked: , {var_check_down.get()}').pack()


## get Enovia page, get home page

In [177]:
### get Enovia page
def get_home_page():
    url = "https://3dx.kla-tencor.com/3dspace/common/KLA_emxNavigator.jsp?" 
    global driver, actions
    try:
        driver = webdriver.Edge()
        driver.set_window_size(1000, 600)
        actions = ActionChains(driver)
    except:
        tk.messagebox.showinfo(title='Webdriver Error',\
            message='Can not get Web Driver, need to update ')
    
    try:
        print('try get url')
        driver.get(url)
        print('got the url')
        sleep(10)
        main_page_return = driver.current_window_handle
        sleep(10)
        driver.switch_to.window(main_page_return)
        print('get main page to return: ', main_page_return)
        return main_page_return
    except:
        print('except when try get home page driver.current_url:',driver.current_url)
        create_popup('Notification', \
            'If can not load Enovia page, please close Microsoft Edge and run again', 10000)
        print('return error at get url')
        return

def get_main_page():
    global main_page
    try:
        driver.switch_to.window(main_page)
    except:
        main_page = get_home_page()
    driver.switch_to.window(main_page)

## def open_find_down

In [178]:
def open_find_down(main_page, part_find):
    
    print('switching to main page')
    driver.switch_to.window(main_page)
    if len(driver.window_handles) > 1:
        for windown in driver.window_handles:
            if windown != main_page:
                driver.switch_to.window(windown)
                driver.close()
    driver.switch_to.window(main_page)
    print('switched')
#     try:
    btn_plus = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, '//*[@id="topbar"]\
    /div[5]/div[2]/div[3]')))
    btn_plus.click()
    btn_report = driver.find_element(By.XPATH, "//*[contains(text(), 'Reports')]") 
    btn_wub = driver.find_element(By.XPATH, "//*[contains(text(), 'Where-Used BOM Report')]")

    actions.move_to_element(btn_report).move_to_element(btn_wub).click().perform()


    # implosion level select 10
    driver.switch_to.window(driver.window_handles[-1])
    wub_win = driver.current_window_handle
    driver.switch_to.frame('pagecontent')
    imp_lvl = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                    "/html/body/form/table[1]/tbody/tr[4]/td[2]/input")))
    imp_lvl.clear()
    imp_lvl.send_keys('15')    
    driver.switch_to.window(wub_win)
    driver.switch_to.frame('pagecontent')
    find_3dot = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                    "/html/body/form/table[1]/tbody/tr[1]/td[2]/input[2]"))).click()
    sleep(random.randint(0, 1))

    driver.switch_to.window(driver.window_handles[-1])
    search_win = driver.current_window_handle
    driver.switch_to.frame('searchBody')
    
    print('Part find: ',part_find, str(part_find).split()[0].split('_')[0][:-1].strip() + '*')
    find_input = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, "/html/body/form/table/tbody/tr[2]/td[2]/input")))
    if '-' not in str(part_find).split()[0].split('_')[0]:
        key_search = str(part_find).split()[0].split('_')[0][:-1].strip() + '*'
    else:
        key_search = str(part_find).split()[0].split('_')[0][:-len(str(part_find).split()[0].split('_')[0].split('-')[-1])-1] + '*'
    print('key_search: ',key_search)
    find_input.send_keys(key_search)
    sleep(random.randint(0, 1))

    #     driver.switch_to.window(find_page)
    driver.switch_to.window(search_win)
    find_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, '//*[@id="divPageFoot"]/form/table/tbody/tr/td[2]/table/tbody/tr/td[2]/a/button')))
    find_btn.click()
    sleep(random.randint(0, 1))

    driver.switch_to.window(driver.window_handles[-1])
    driver.switch_to.frame('pagecontent')

    parts = driver.find_elements(By.XPATH, '//*[@id="UITable"]/tbody/tr')

    # part_pattern = re.compile(r'\b\d*[A-Z]*\d+[-]*\d*[A-Z]*\d+[-]*[A-Z]*\d+[-]*[A-Z]*\d+\b')
    i = len(parts)
    while i > 0:
        i -= 1

        pn_name = driver.find_element(By.XPATH, '/html/body/form/table/tbody/tr[' + str(i+1) + ']/td[2]')
        state = driver.find_element(By.XPATH, '/html/body/form/table/tbody/tr[' + str(i+1) + ']/td[5]')

        if pn_name.text.strip() == str(part_find).strip().split()[0].split('_')[0] \
            and state.text.lower() == 'release':
            part_select = driver.find_element(By.XPATH, '//*[@id="UITable"]/tbody/tr[' + str(i+1) + ']/td[1]/input')
            sleep(random.randint(0, 1))
            part_select.click()
            break
        elif pn_name.text.strip() == str(part_find).strip().split()[0].split('_')[0][:-1] \
             + str(int(str(part_find).strip().split()[0].split('_')[0][-1])-1)\
            and state.text.lower() == 'release':
            part_select = driver.find_element(By.XPATH, '//*[@id="UITable"]/tbody/tr[' + str(i+1) + ']/td[1]/input')
            sleep(random.randint(0, 1))
            msg_select_confirm = tk.messagebox.showinfo('Select PN',\
            'PN: ' + str(part_find).strip().split()[0].split('_')[0] + ' has not yet release, Please select, then click OK',\
            icon='question')
            break

        elif pn_name.text.strip() == str(part_find).strip().split()[0].split('_')[0][:-1] \
             + str(int(str(part_find).strip().split()[0].split('_')[0][-1])+1)\
            and state.text.lower() == 'release':
            part_select = driver.find_element(By.XPATH, '//*[@id="UITable"]/tbody/tr[' + str(i+1) + ']/td[1]/input')
            sleep(random.randint(0, 1))
            msg_select_confirm = tk.messagebox.showinfo('Select PN',\
            'PN: ' + str(part_find).strip().split()[0].split('_')[0] + ' has released a newer version, Please select, then click OK',\
            icon='question')
            break

        
    sleep(1)
    if i == 0:
        print('can not find WUB for ',part_find)
        print('go to next part find')
        return

    #select correct part and submit
    driver.switch_to.window(driver.window_handles[-1])
    submit_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, '//*[@id="bottomCommonForm"]/table/tbody/tr/td[2]/table/tbody/tr/td[2]/a/button')))
    submit_btn.click()
    sleep(random.randint(0, 2))

    # click done to find WUB
    driver.switch_to.window(driver.window_handles[-1])
    done_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, '//*[@id="bottomCommonForm"]/table/tbody/tr/td[2]/table/tbody/tr/td[2]/a/button')))
    done_btn.click()
    sleep(random.randint(0, 2))

    # select export to excel file
    driver.switch_to.window(driver.window_handles[-1])
    try:
        print('try to get excel file')
        to_excel =  WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                    '//*[@id="ktemxWhereUsedBOMReport_command_0"]/img')))
    except TimeoutException:
        print('can not find excel file to download')

    driver.switch_to.frame('pagecontent')
    tables = driver.find_elements(By.XPATH, '//*[@id="UITable"]/tbody/tr')
    driver.switch_to.window(driver.window_handles[-1])
    if len(tables) > 2:
        print('where used BOM has parent',len(tables))
        sleep(1)
        to_excel.click()      
        sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        driver.switch_to.frame('pagecontent')
        down_file = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/table/tbody/tr/td[2]/a')))
        sleep(1)
        down_file.click()
        print('has click to download')
        sleep(2)
        driver.close()
    #     driver.switch_to.window(wub_page)
        driver.switch_to.window(driver.window_handles[1])
    else:
        print('where used BOM has not contents')
    driver.close()

## def from_request

In [179]:
def from_request(part_list,down_folder, main_page):
# ##    part_list = request["Part"].dropna().tolist()
    
    part_list_not_down = []
    part_list_not_down.append("Part still not download")
    print('delete files in download folder')
    for part in part_list:
#         part = cur_part[0]
    # delete old file
        
        for file in os.listdir(down_folder):
            if fnmatch.fnmatch(file, '*.xlsx') \
            and fnmatch.fnmatch(file, '*' + str(part).split()[0].split('_')[0][:-3] + '*'):
                del_part = down_folder + '\\' + file
                os.remove(del_part)
                print('deleted', file, 'from', down_folder)
        # down new file
    for part in part_list:
        part = str(part).split()[0].split('_')[0]
        print('go to download', part,type(part),str(part)[:-1])
        
        try:
            
            open_find_down(main_page, part)
        except:
            mes = 'can not down WUB for part ' + part
        #         if a <= 20:
            create_popup('Download error', mes, 10000)
            found = False
            for item in part_list_not_down:
                if part[:-2] in item:
                    found = True
                    break
            if found == False:
                part_list_not_down.append(part)
            continue


    loop = True
    r = 1
    while loop and r < 20:
        for part in part_list:
            found = False
            for new_file in os.listdir(down_folder):
                if fnmatch.fnmatch(new_file, '*.xlsx') \
                and fnmatch.fnmatch(new_file, '*' + str(part).split()[0].split('_')[0][:-3] + '*'):
                    found = True
                    break
            if found:
                loop = False
            else:
                print(part, 'still in downloading, waited', r, 'times')
                loop = True
                r += 1
                sleep(1)
                break

    if r >= 20 and loop:
        print('WUBs download is still not completed, the script may be not fully')    
        mes = 'WUBs download is still not completed, the script may be not fully'
        thread = threading.Thread(target = create_popup, \
            args = ('Parts can not download', mes, 5000))
        thread.start()

    print('copy new WUB to the selected folder')
    for part in part_list:
    #         print('part',part)
        for new_file in os.listdir(down_folder):
        #             print('file in download folder',new_file)
            if fnmatch.fnmatch(new_file, '*.xlsx') \
            and fnmatch.fnmatch(new_file, '*' + str(part).split()[0].split('_')[0][:-3] + '*'):
                for file_in_req in os.listdir(file_path):
                    if fnmatch.fnmatch(file_in_req, '*.xlsx') \
                    and fnmatch.fnmatch(file_in_req, '*' + str(part).split()[0].split('_')[0][:-3] + '*'):
                        del_p2 = file_path + '\\' + file_in_req
                        os.remove(del_p2)
                        print(f'PN {part} file {file_in_req} has been removed')
                        break
                full_file_name = down_folder + '\\' + new_file
                shutil.copy(full_file_name, file_path)
                print('copied', new_file, 'to', file_path)
                break
    mes = "\n".join([str(line).replace("'","").replace("[","").replace("]","")\
                    for line in part_list_not_down])
    thread = threading.Thread(target = create_popup, \
        args = ('Parts can not download: ', mes, 50000))
    thread.start()


## def txt_list

In [180]:
def txt_list(parent,children,out_txt, num, letter,n):
    print('txt_list def input:',parent,children,'out_txt',out_txt,'num', num,'letter', letter,'n',n)
    
    if len(children) == 0:
        print(parent, 'part without children')
        if num < n + 1:
            num += 1

        for par in parent:
            if not any(str(par[2]) in line for line in out_txt):
                
#                 continue
#             else:
                print(f' par go to add {par}')
                out_txt.append(f'{num}{letter}. {par[2]} ({par[4]}) is {par[5]} to -{par[6]}')
            else:
                print(f'par {par} has exist in out_txt')
        out_txt.append('')
        if letter == 'z':
            letter = 'a'
        else:
            letters = string.ascii_letters
            letter = letters[letters.index(letter) +1] 
    elif len(parent) == 1:
        if num < n + 1:
            num += 1
        out_txt.append(f'{num}{letter}. {parent[0][2]} ({parent[0][4]}) is {parent[0][5]} to -{parent[0][6]} for below BOM change:') 
        for child in children:
            out_txt.append(f'--> {child[2]} ({child[4].strip()}) is {child[5]} to -{child[6]}')
        out_txt.append('')
        if letter == 'z':
            letter = 'a'
        else:
            letters = string.ascii_letters
            letter = letters[letters.index(letter) +1] 
    else: #if len(children) == 1: 
        if num < n + 1:
            num += 1
        out_txt.append(f'{num}{letter}. Below parent changed to reflect {children[0][2]} ({children[0][4].strip()}) is {children[0][5]} to -{children[0][6]}:') 
        for par in parent:
            out_txt.append(f'--> {par[2]} ({par[4].strip()}) is {par[5]} to -{par[6]}') 
        out_txt.append('')
        if letter == 'z':
            letter = 'a'
        else:
            letters = string.ascii_letters
            letter = letters[letters.index(letter) +1] 
    return out_txt, num, letter

## def find_parent

In [181]:
def find_par(child_lst, par_lst, lst_items):
    for child in child_lst:
        loop_index = child[8]
        child_lev = int(child[1])
        child_req = child[0]
        while loop_index + 1 < len(lst_items):

            loop_index = loop_index + 1
            if int(lst_items[loop_index][1]) >= child_lev:
                break

            if int(lst_items[loop_index][1]) == child_lev - 1\
            and lst_items[loop_index][0] == child_req and lst_items[loop_index][7] != 'done':

                lst_items[loop_index][7] = 'done'
                if not any(str(lst_items[loop_index][2]) in par for par in par_lst): #lst_items[m] not in child_lst:
                    par_lst.append(lst_items[loop_index])
    return par_lst

## def extract_path of ECO request form

In [182]:
def extract_path():
    global file_full, file_type, file_path, file_name, down_folder
    file_full = entry_req_form.get().strip()
    if file_full =='':
        print('file none, exit program')
        get_form_file()
        file_full = entry_req_form.get().strip()
        # sys.exit()
    file_type = os.path.splitext(file_full)[1]
    file_path = os.path.dirname(file_full)
    file_name = os.path.basename(file_full)
    down_folder = os.path.expanduser('~') + '\Downloads'

## def read_excel

In [183]:
def read_excel(file_full):
    global request
    try:
        request = pd.read_excel(file_full, usecols = 'A:O')
        request.columns = ["Part","Description", "New/Existing",\
                           "CCB","Cost","Qty",\
                           "PO", "KT", "KT Mfg.","Field Inventory", "Installed Base",\
                           "Field Returns","Reason","different","Notes"]
        request = request[9:]
        request = request[request['Part'].notna()]
    except:
        request = []
        

## def get_cur_list

In [184]:
def get_cur_list():
    global cur_pn_list
    cur_pn_list = []
    
    for i in range(len(request)):
        
        # find up to from request
        if '/' in request.iloc[i,2]:
            left_pn = request.iloc[i,2].split('/')[0].strip().split()[0].split('_')[0]
            right_pn = request.iloc[i,2].split('/')[-1].strip().split()[0].split('_')[0]
            left_rev = request.iloc[i,2].split('/')[0].strip().split()[-1].split('_')[-1]
            if len(left_rev) > 3 or left_rev[0].isalpha() == False:
                left_rev = '........Error 001: can not find left_rev.'
            right_rev = request.iloc[i,2].split('/')[-1].strip().split()[-1].split('_')[-1]
            if len(right_rev) > 3 or right_rev[0].isalpha() == False:
                right_rev = '........Error 002: not found right_rev.'
            print(i,'left_pn:',left_pn,left_rev,'/ right_pn:',right_pn,right_rev)
            
            
    #         cur_pn = request.iloc[i,0].strip().split()[0].split('_')[0]
            cur_pn = right_pn
            if cur_pn.split('-')[-1].isnumeric() and left_pn.split('-')[-1].isnumeric():
                if int(cur_pn.split('-')[-1]) > int(left_pn.split('-')[-1]):
                    cur_pn = left_pn
            else:
                print(cur_pn,left_pn,'The Dash rolled is not numbers')
    #         elif int(cur_pn[:-1]) > int(right_pn[:-1]):
    #             cur_pn = left_pn
            if cur_pn[:-3] != str(request.iloc[i,0]).strip().split()[0].split('_')[0][:-3]:
                print(f"Part { str(request.iloc[i,0]).strip().split()[0].split('_')[0]} \
                in A column not match with C column {cur_pn}")
            cur_dash_roll = cur_pn.split('-')[-1]
            cur_rev = ''
            if len(cur_dash_roll) > 3 or not cur_dash_roll.isnumeric():
                print('...Error 003: Can not find Current rolled')
                cur_dash_roll = ''
            
            
            if left_pn == cur_pn and right_pn == cur_pn:
                print(f' left/right pn {left_pn}/{right_pn}')
                rolled = 'Rev rolled'
                
                #find revision
                print('len(right_rev)',len(right_rev))
                if len(right_rev) < 5 and len(right_rev) > 1:
                    cur_rev = right_rev
                    letters = string.ascii_uppercase
                    print('letters.index(cur_rev[0])',letters.index(cur_rev[0]))
                    if len(left_rev) < 5 and len(left_rev) > 1:
                        if letters.index(cur_rev[0]) > letters.index(left_rev[0]):
                            cur_rev = left_rev
                        elif letters.index(cur_rev[-1]) > letters.index(left_rev[-1]):
                            cur_rev = left_rev
        #         next_char = letters[index +2] + 'A'  
        #         cur_rev = request.iloc[i,0].strip().split()[-1].split('_')[-1]
                print('cur_pn',cur_pn,'cur_rev',cur_rev)
        #         print(cur_rev[0].isalpha())
                if len(cur_rev) > 3 or cur_rev == '' or cur_rev[0].isalpha() == False:
                    cur_rev = '...Error 004: can not find current PN Rev'
                print('cur_pn:',cur_pn,'cur_rev:',cur_rev)
            cur_pn_list.append(cur_pn + "_" + cur_rev)
        else:
            print(f' can not find New/Existing pn for {i} - {cur_pn}')
            continue
    # return cur_pn_list

## def down_wub

In [185]:
def down_wub():
    error = False
    driver.switch_to.window(main_page)
    try:
        print('try to get button +')
        btn_plus = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,\
                                                        '//*[@id="tvc-bin-icon"]/i')))
        print('got button + at the first try')
        sleep(random.randint(2, 3))
    except TimeoutException:
        print('can not catch the button, need to refresh the page')
        driver.refresh()
        print('refreshed')
        sleep(random.randint(5, 7))
        try:
            print('try second time to get button +')
            btn_plus = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                            '//*[@id="tvc-bin-icon"]/i')))
            print('got button + after refresh the page')
            sleep(random.randint(2, 3))
        except:
#                 print('can not catch the button, need to refresh the page')
#                 tk.messagebox.showinfo(title='Message', \
#                 message='Can not go to report/whereusedbom, check your connection and try again')
#                 error = True
            tk.messagebox.showinfo(title='Download Error',
            message='Can not get "+" button, please try again?')
#             sys.exit()
            error = True
            return error
#                 print('return error at get +')
            # return error
#                 print('check return at get +')
#             sys.exit()
#         return error

    ### down WUB
    # if le(cur_pn_list) > 1:
    try:
        
        print('try down WUBs from request')
        from_request(cur_pn_list,down_folder, main_page)
        print('executed from_request')
    except:
        tk.messagebox.showinfo(title='Download Error',
            message='Can not download WhereUsedBOMs for items, please try again?')
        print('E1000 down_wub: try down from request')
        error = True
        return error


## def get_project_key

In [186]:
def get_project_key():
    global plant_pro_key
    plant_pro_input = entry_plant_pro_key.get().strip()
    if plant_pro_input is not None and plant_pro_input != '':
        newList = re.split(',|_|\.|/|\*', plant_pro_input)
        # newList = newList.split('*')
        plant_pro_key = [s.strip() for s in newList if s != '']
        print('plant_pro_key: ',plant_pro_key)
        
    else:
        plant_pro_key = ['']
# get_project_key()

## def fun_up_to

In [187]:
def fun_up_to(rolled,cur_pn,cur_rev):
    up_to = ''
#     if 'Dash' in lst_items[i][5] and len(lst_items[i][5]) < 15 and len(lst_items[i][6]) < 5
    if 'clone' in rolled.lower() and len(rolled) < 15 : # == 'Dash rolled':
        up_to = '***'
    elif 'dash' in rolled.lower() and len(rolled) < 15 : # == 'Dash rolled':
            cur_dash_in_list = cur_pn.split('-')[-1]
#             print('cur_dash_in_list.isnumeric()',cur_dash_in_list.isnumeric())
            if len(cur_dash_in_list) < 2 or len(cur_dash_in_list) > 4 or not cur_dash_in_list.isnumeric():
                up_to = '(Error 501: Not found roll)'
            else:
                dash_rolled = int(cur_dash_in_list) + 1
                up_to = '{:03d}'.format(dash_rolled)
    elif 'rev' in rolled.lower() and (len(rolled) < 15 or 'goto' in rolled): # == 'Rev rolled':
        print('rev rolled')
        rev = cur_rev
        letters = string.ascii_uppercase
        if rev.isalpha():
            if rev[-1] == 'Y':
                if rev[0] == 'H' or rev[0] == 'N' or rev[0] == 'P'\
                          or rev[0] == 'R' or rev[0] == 'W':
                    index = letters.index(rev[0])
                    next_char = letters[index +2] + 'A'   
                else:
                    index = letters.index(rev[0])
                    next_char = letters[index +1] + 'A'   
            elif rev[-1] == 'H' or rev[-1] == 'N' or rev[-1] == 'P'\
                          or rev[-1] == 'R' or rev[-1] == 'W':
                index = letters.index(rev[-1])
                next_char = rev[0] + letters[index +2] 
            else:
                index = letters.index(rev[-1])
                next_char = rev[0] + letters[index +1] 
            up_to = next_char
        elif rev == 'A0' or rev == 'AA0':
            up_to = 'AB'
        else:
            up_to = '(can not collect up to)'
#     else:
#         up_to = 'can not collect up to due to can not define Rolled Type'
    return up_to


## Write Script fun

In [188]:
def write_script():
    # global script_file
    lst_items = []
    emp_fru = []
    # fi = 0
    for i in range(len(request)):
        up_to = ""
        # find up to from request
        if '/' in request.iloc[i,2]:
            left_pn = request.iloc[i,2].split('/')[0].strip().split()[0].split('_')[0]
            right_pn = request.iloc[i,2].split('/')[-1].strip().split()[0].split('_')[0]
            left_rev = request.iloc[i,2].split('/')[0].strip().split()[-1].split('_')[-1]
            if len(left_rev) > 3 or left_rev[0].isalpha() == False:
                left_rev = '........Error 001: can not find left_rev.'
            right_rev = request.iloc[i,2].split('/')[-1].strip().split()[-1].split('_')[-1]
            if len(right_rev) > 3 or right_rev[0].isalpha() == False:
                right_rev = '........Error 002: not found right_rev.'
            print(i,'left_pn:',left_pn,left_rev,'/ right_pn:',right_pn,right_rev)
    
            cur_pn = right_pn
            if cur_pn.split('-')[-1].isnumeric() and left_pn.split('-')[-1].isnumeric():
                if int(cur_pn.split('-')[-1]) > int(left_pn.split('-')[-1]):
                    cur_pn = left_pn
            else:
                print(cur_pn,left_pn,'The Dash rolled is not numbers')
    
            if cur_pn[:-3] != str(request.iloc[i,0]).strip().split()[0].split('_')[0][:-3]:
                print(f"Part { str(request.iloc[i,0]).strip().split()[0].split('_')[0]} \
                in A column not match with C column {cur_pn}")
            cur_dash_roll = cur_pn.split('-')[-1]
            cur_rev = ''
            if len(cur_dash_roll) > 3 or not cur_dash_roll.isnumeric():
                cur_dash_roll = ''
    
            if left_pn == cur_pn and right_pn == cur_pn:
                rolled = 'Rev rolled'
                
                #find revision
                if len(right_rev) < 5 and len(right_rev) > 1:
                    cur_rev = right_rev
                    letters = string.ascii_uppercase
                    if len(left_rev) < 5 and len(left_rev) > 1:
                        if letters.index(cur_rev[0]) > letters.index(left_rev[0]):
                            cur_rev = left_rev
                        elif letters.index(cur_rev[-1]) > letters.index(left_rev[-1]):
                            cur_rev = left_rev
    
                if len(cur_rev) > 3 or cur_rev == '' or cur_rev[0].isalpha() == False:
                    cur_rev = '...Error 004: can not find current PN Rev'
    
                up_to = fun_up_to(rolled,cur_pn,cur_rev)
    
            elif cur_pn == left_pn and cur_pn != right_pn:
                rolled = 'Dash rolled'
                new_dash = right_pn.split('-')[-1]
                up_to = fun_up_to(rolled,cur_pn,cur_rev)
                if str(new_dash[-1]) != str(up_to[-1]) or len(new_dash) > 3 or not new_dash.isnumeric(): #1
                    up_to =  up_to +'/'+ new_dash + ' (select)'
            elif cur_pn != left_pn and cur_pn == right_pn:
                rolled = 'Dash rolled'
                new_dash = left_pn.split('-')[-1]
                up_to = fun_up_to(rolled,cur_pn,cur_rev)
                if str(new_dash[-1]) != str(up_to[-1])  or len(new_dash) > 3 or not new_dash.isnumeric(): #1
                    up_to =  up_to +'/'+ new_dash + ' (select)'
            else:
                up_to = '..........Error 110: Can not find up to'
        else:
            continue
        if type(request.iloc[i,1]) == float or len(request.iloc[i,1]) < 5 :
            print('request.iloc[i,1] = ""')
            request.iloc[i,1] = 'Check Des.'
        lst_items.append([i, 0, cur_pn, cur_rev,\
                        request.iloc[i,1],rolled, up_to, '-' , '--','---','----','-----',plant_pro_key])
    
        child_roll = rolled
        req_item = str(request.iloc[i,0]).split()[0][:7]
        found = False
        for file in os.listdir(file_path):
            if fnmatch.fnmatch(file, '*.xlsx') and fnmatch.fnmatch(file, '*' + req_item + '*'):
                found = True
                sub_df = pd.read_excel(file_path + '/' + file, usecols = 'A:O')[9:]
                sub_df.insert(loc=0, column='A', value=i)
                sub_df.columns = ["request","level","part", "rev",\
                                   "des","type","state", "fru",\
                                   "packeged", "pro", "non","sap", "plant",\
                                   "procurement","special","bulk"]
                sub_df = sub_df.fillna(0)
                sub_lst = sub_df[['request','level','part','rev','des','type',\
                                  'state','fru','procurement','plant','pro','sap','non']].values.tolist()
    #                 print(*sub_lst, sep = '\n')
                break
        if found == False:
            continue
        print('initial child_roll',child_roll)
        for j in range(len(sub_lst)-1,-1,-1):
    #         print(sub_lst[j][2],'is',str(sub_lst[j][11]))
            if any(word in  str(sub_lst[j][11]).lower() for word in plant_pro_key)\
            or any(word in  str(sub_lst[j][10]).lower() for word in plant_pro_key):
    #             print(sub_lst[j][2],'found _________ key:',str(sub_lst[j][11]))
                sub_lst[j][12] = plant_pro_key
            if str(sub_lst[j][9]).isnumeric() and int(sub_lst[j][9]) > 29:
    #             print('sub_lst[j][9]',sub_lst[j][9])
                sub_lst.pop(j)
        
        
        for j in range(len(sub_lst)):
            sub_lst[j][0] = i
            sub_lst[j][5] = ''
            sub_lst[j][6] = ''
            if sub_lst[j][7] == 0 or sub_lst[j][8] == 0:
                if not any(sub_lst[j][2] in fru for fru in emp_fru):
                    emp_fru.append(sub_lst[j][2])
                    
    
        # find rolled type
        min_lev = min(int(l[1]) for l in sub_lst)
        top_most = -min_lev
            
        # loop from min level to find FRU part or Buy part
        for j in range(top_most,-1,-1):
            for k in range(len(sub_lst)):
                if sub_lst[k][12] == 0:
    #                 print('j/k',j,'/',k,sub_lst[k][2],'is not match key',sub_lst[k][10])
                    continue
                elif int(sub_lst[k][1]) == -j\
                and (sub_lst[k][7]== 'Yes' or sub_lst[k][8]== 'F'):
                    sub_lst[k][5] = rolled
                    w = j
                    h = k
                    while w > -1 and h > -1:
                        h = h - 1
    #                     print('check________________',h,sub_lst[h][3],sub_lst[h][12],sub_lst[h][12] != 0)
                        if int(sub_lst[h][1]) == -w +1:
                            if sub_lst[h][12] != 0 and sub_lst[h][5] != rolled:
                                sub_lst[h][5] = rolled
        #                         elif sub_lst[h][12] == 0:
        #                             sub_lst[h][5] = rolled.split()[0] + ' Clone'
    #                         w -= 1
                            break
                        if int(sub_lst[h][1]) > -w +1:
                            break
    
        # loop from level 0, to find dash/rev rolled
        for j in range(1,-min_lev + 1):
            # loop all item in WUB
            for k in range(len(sub_lst)):

                if int(sub_lst[k][1]) == -j and sub_lst[k][12] != 0:
                
                    # Loop to find child of part in current level
                    found_child = False
                    for l in range(len(sub_lst),0,-1):
                        if int(sub_lst[l-1][1]) == -j+1 and l - 1 < k and sub_lst[l-1][12] != 0:
                            child_roll = sub_lst[l-1][5]
                            found_child = True
                            break
                    
                    if found_child == False and j > 1:                    
                        child_roll = ''
    #                     sub_lst[k][10] = -1
                        sub_lst[k][5] = ''
                    if sub_lst[k][5] == '':
                        if child_roll == '':
                            sub_lst[k][5] = ''
                        elif type(sub_lst[k][7]) != str or type(sub_lst[k][8]) != str\
                            and sub_lst[k][12] != 0:
                            # if child dash roll but can not find current part's Pro. or FRU --> goto Rev
                            if 'dash' in str(child_roll).lower() and len(child_roll) < 15:
                                sub_lst[k][5] = 'Procurement or FRU is not string, goto Rev roll'
    #                             print('type of',type(sub_lst[k][7]))
                            # else child rev roll go to nothing
                            else:
                                sub_lst[k][5] = 'Procurement or FRU is not string'
                        elif 'dash' in str(child_roll).lower()  and len(child_roll) < 15\
                            and sub_lst[k][12] != 0: # == 'dash rolled':
        #                     print('sub_lst[k][9]',sub_lst[k][9])
                            if str(sub_lst[k][8]).lower() == 'f' or sub_lst[k][7].lower() =='yes':
                                sub_lst[k][5] = 'Dash rolled'
    #                         elif str(sub_lst[k][8]).lower() == 'e' and sub_lst[k][7].lower() =='yes':
    #                             sub_lst[k][5] = 'Dash rolled'
                            elif str(sub_lst[k][8]).lower() == 'e' and sub_lst[k][7].lower() =='no':
                                sub_lst[k][5] = 'Rev rolled'
                            else:
    #                             if 'dash' in str(child_roll).lower():
                                sub_lst[k][5] = 'Missing Procurement or FRU info. Maybe Zdoc file, goto Rev rolled'
    #                                 print('type of',type(sub_lst[k][7]))
    #                             else:
    #                                 sub_lst[k][5] = 'Missing Procurement or FRU info.'
    #                             sub_lst[k][5] = 'Missing Procurement or FRU info. Maybe Zdoc file, goto Rev rolled'
    #                             print('can not find the rolled')
                        elif 'rev' in str(child_roll).lower() and len(child_roll) < 15\
                            and sub_lst[k][12] != 0: # == 'rev rolled':
                            if str(sub_lst[k][8]).lower() == 'f' or sub_lst[k][7].lower() =='yes':
                                sub_lst[k][5] = 'Rev rolled'
    #                         elif str(sub_lst[k][8]).lower() == '0':
    #                             sub_lst[k][5] = 'Missing Procurement, Maybe Zdoc file, goto Rev rolled'
                            else:
                                sub_lst[k][5] = ''
                    if var_check_plant.get() == 1 and str(sub_lst[k][9]).isnumeric() and int(sub_lst[k][9]) < 6\
                        and sub_lst[k][5] == 'Dash rolled':
                        sub_lst[k][5] = 'Rev rolled'
    
        dif = 0
        for item in lst_items:
            found_dif = False
            for sub_item in sub_lst:
                if item[2] == sub_item[2]:
                    if item[1] != sub_item[1]:   
                        dif = int(item[1]) - int(sub_item[1])
    #                         print('difference level:',dif,'from',sub_item,'\n',item)
                    found_dif = True
                    break
            if found_dif:
                break        
        lst_items = lst_items + sub_lst
        if dif != 0:        
            for j in range(len(lst_items)-len(sub_lst)-1,len(lst_items)):
                lst_items[j][1] = int(lst_items[j][1]) + dif
                
    
    
    # correct des
    for c in range(len(lst_items)):
        for d in range(len(request)):
            if (lst_items[c][2][:7] == str(request.iloc[d,0])[:7])\
            and lst_items[c][4] != request.iloc[d,1]:            
                print(lst_items[c][4],'not match with',request.iloc[d,1],'collect from Request form')
                lst_items[c][4] = request.iloc[d,1]
    
    
    # correct up rolled and part_dash_rev_rolled
    for item in lst_items:
        for sub_item in lst_items:
            if item[2][:7] == sub_item[2][:7] and item[12] !=0 and sub_item[12] !=0:
    #                 print(item[2][:7],sub_item[2][:7])
                # correct Rolled up
                if item[5] == 'Dash rolled' and sub_item[5] == 'Rev rolled':
                    sub_item[5] = 'Dash rolled'
                elif item[5] == 'Dash rolled' and sub_item[5] == '':
                    sub_item[5] = 'Dash rolled'
                elif item[5] == 'Rev rolled' and sub_item[5] == 'Dash rolled':
                    item[5] = 'Dash rolled'
                elif item[5] == 'Rev rolled' and sub_item[5] == '':
                    sub_item[5] = 'Rev rolled'
                elif item[5] == '' and sub_item[5] == 'Dash rolled':
                    item[5] = 'Dash rolled'
                elif item[5] == '' and sub_item[5] == 'Rev rolled':
                    item[5] = 'Rev rolled'
    
                # correct part name, dash rolled value
                if item[2][8:11] != sub_item[2][8:11]:
                    if item[2][8:11] > sub_item[2][8:11]:
    #                         print(sub_item[2],'need to up',item[2])
                        sub_item[2] = item[2]
                        sub_item[3] = item[3]
                        sub_item[4] = item[4]
                    elif item[2][8:11] < sub_item[2][8:11]:
    #                         print('item:',item[2],'need to up',sub_item[2])
                        item[2] = sub_item[2]
                        item[3] = sub_item[3]
                        item[4] = sub_item[4]
                
                
    # find clone PN, if not in consider plant
    min_lev = min(int(l[1]) for l in lst_items)
    top_most = -min_lev
    
    # loop from min level to find FRU part or Buy part
    for j in range(top_most,-1,-1):
        for k in range(len(lst_items)):
            if lst_items[k][12] == 0:
    #                 print('j/k',j,'/',k,sub_lst[k][2],'is not match key',sub_lst[k][10])
                continue
            elif int(lst_items[k][1]) == -j\
            and lst_items[k][5] != '':
    #             lst_items[k][5] = rolled
                w = j
                h = k
                cur_request = lst_items[k][0]
                while w > -1 and h > -1:
                    h = h - 1
    #                     print('check________________',h,sub_lst[h][3],sub_lst[h][12],sub_lst[h][12] != 0)
                    if int(lst_items[h][1]) == -w +1:
                        if lst_items[h][12] == 0:
                            print('correct________________',h,lst_items[h][3],lst_items[h][12],lst_items[h][12] != 0)
        #                     if lst_items[h][5] != rolled:
                            lst_items[h][5] = "Need a clone"
                        break
                    if int(lst_items[h][1]) > -w +1 or lst_items[h][0] != cur_request:
                        break            
    
    for i in range(len(lst_items)):
        if len(lst_items[i][6]) < 5:
            lst_items[i][6] = fun_up_to(lst_items[i][5],lst_items[i][2],lst_items[i][3])
        
    
    print('sub_lst',*lst_items, sep = '\n')
    main_df = pd.DataFrame(lst_items,columns =\
            ['Request','Level','PN','Rev','Des.','Roll','up_to','FRU',\
             'Procurement','Plant Status','Project','Sap Plant','Not considered'])
    main_df.to_csv(file_path + '\\Result_reporting.csv')
    for k in range(len(lst_items)-1, -1, -1):
        if lst_items[k][6] == '':
            lst_items.pop(k)
    #     df_roll = pd.DataFrame(lst_items,columns =['request','level','part','rev','des','roll','up_to','fru','procurement'])
    #     df_roll.to_csv(file_path + '\\out_roll_only.csv')
    for i in range(len(lst_items)):
        lst_items[i][8] = i
    
    # # ----------------write script and export to text file-------------------------
    # if len(lst_items) > 0:
    #     max_req = max(l[0] for l in lst_items)
    #     max_lev = max(int(l[1]) for l in lst_items)
    #     min_lev = min(int(l[1]) for l in lst_items)
    
    # out_txt = []
    # num = 1
    
    # #loop request
    # n_increase = 0
    # for n in range(max_req + 1):
    #     print('1-1110 n/increase', n,n_increase)
    #     letter = "a"
    #     increase = False
    #     #loop level
    #     for i in range(min_lev,max_lev + 1):
    #         #loop items
    #         for j in range(len(lst_items)):
    #             par_lst = []
    #             child_lst = []
    
    #             if int(lst_items[j][1]) == i and lst_items[j][0] == n and lst_items[j][7] != 'done':  
    #                 par_lst.append(lst_items[j])
    #                 lst_items[j][7] = 'done'
                    
    #                 # check part request is only or not, need to find from other request
    #                 one_part = True
    #                 for k in range(len(lst_items)):
    #                     if lst_items[k][0] == lst_items[j][0] and k != j:# and lst_items[k][1] == lst_items[j][1]:
    #                         one_part = False
    #                         break
    # #                 print('one_part',one_part)
    #                 if one_part:
    #                     # find in other request if have, break, if not, write line
    #                     pn_in_other_req = False
    #                     for l in range(len(lst_items)):
    # #                         print('find other request',lst_items[j][2][:7],lst_items[l][2][:7])
    #                         if lst_items[l][0] != n and lst_items[j][2][:7] == lst_items[l][2][:7]:
    #                             pn_in_other_req = True
    #                             break
    #                     if pn_in_other_req:
    #                         break
    #                     else:
    #                         increase = True
    #                         out_txt , num, letter = txt_list(par_lst,child_lst,out_txt, num, letter,n_increase)
    #                         print(n,i,j,'out_txt num and letter of one part:',out_txt , num, letter)
    #                         break
    
    #                 # break if part is the first of request, or not continue 
    #                 if lst_items[j][0] != lst_items[j-1][0] or lst_items[j][8] != lst_items[j-1][8] + 1:
    #                     break
    #                 # find child
    #                 for k in range(len(lst_items)-1,-1,-1):
    #                     if int(lst_items[k][1]) == int(lst_items[j][1]) + 1\
    #                         and k < j and lst_items[k][0] == n:
    #                         child_lst.append(lst_items[k])
    #                         break                
    
    # #                 find same parent and child of the same parent of other request
    #                 for l in range(len(lst_items)):
    #                     if lst_items[l][2] == lst_items[j][2] and l != j and lst_items[l][7] != 'done':
    #                         lst_items[l][7] = 'done'
    
    #                         for m in range(len(lst_items)-1,-1,-1):   
    #                             if int(lst_items[m][1]) == int(lst_items[l][1]) + 1\
    #                             and m < l and lst_items[m][0] == lst_items[l][0]:
    #                                 if not any(lst_items[m][2] in child for child in child_lst): #lst_items[m] not in child_lst:
    #                                     child_lst.append(lst_items[m])
    #                                 break
    
    #                 # find same childrent level and collect parent, try if child = 1
    #                 if len(child_lst) == 1:
    #                     par_lst = find_par(child_lst, par_lst, lst_items)
    #                     for k in range(len(lst_items)):
    #                         if lst_items[k][2] == child_lst[0][2] and k != child_lst[0][8]:
    #                             child_lst1 = []
    #                             child_lst1.append(lst_items[k])
    #                             par_lst = find_par(child_lst1, par_lst, lst_items)
    #                 print('1-1111 n and num and letter', n_increase, num, letter)
    #                 increase = True
    #                 out_txt , num, letter = txt_list(par_lst,child_lst,out_txt, num, letter,n_increase)
    #                 print('1-1111 n and num and letter return', n_increase, num, letter)
    #     if increase:
    #         print('1-1112 increase at n',n_increase,n)
    #         n_increase +=1

    out_txt = get_out_txt(lst_items)
    
    with open(file_path + '\\' + str(date[2:10]) + '_script_auto.txt', 'w') as f:
        for line in out_txt:
            f.write(f'{line}\n')
    
    script_file = file_path + '\\' + str(date[2:10]) + '_script_auto.txt'
    if len(script_file) > 3:
        entry_script_file.delete(0,END)
        entry_script_file.insert(0,script_file)
    print('script done')
# write_script()

In [189]:
# ----------------write script and export to text file-------------------------
def get_out_txt(lst_items):
    if len(lst_items) > 0:
        max_req = max(l[0] for l in lst_items)
        max_lev = max(int(l[1]) for l in lst_items)
        min_lev = min(int(l[1]) for l in lst_items)
        print(f'max: {max_lev}, min: {min_lev}')
    out_txt = []
    num = 1
    
    #loop request
    n_increase = 0
    for n in range(max_req + 1):
        # print('1-1110 n/increase', n, n_increase)
        letter = "a"
        increase = False
        #loop level
        for i in range(max_lev, min_lev - 1, - 1):
            #loop items
            for j in range(len(lst_items)):
                par_lst = []
                child_lst = []
    
                if int(lst_items[j][1]) == i and lst_items[j][0] == n and lst_items[j][7] != 'done':  
                    par_lst.append(lst_items[j])
                    lst_items[j][7] = 'done'
                    
                    # check part request is only or not, need to find from other request
                    one_part = True
                    for k in range(len(lst_items)):
                        if lst_items[k][0] == lst_items[j][0] and k != j:# and lst_items[k][1] == lst_items[j][1]:
                            one_part = False
                            break
    #                 print('one_part',one_part)
                    if one_part:
                        # find in other request if have, break, if not, write line
                        pn_in_other_req = False
                        for l in range(len(lst_items)):
    #                         print('find other request',lst_items[j][2][:7],lst_items[l][2][:7])
                            if lst_items[l][0] != n and lst_items[j][2][:7] == lst_items[l][2][:7]:
                                pn_in_other_req = True
                                break
                        if pn_in_other_req:
                            break
                        else:
                            increase = True
                            out_txt , num, letter = txt_list(par_lst,child_lst,out_txt, num, letter,n_increase)
                            print(n,i,j,'out_txt num and letter of one part:',out_txt , num, letter)
                            break
    
                    # break if part is the first of request, or not continue 
                    if lst_items[j][0] != lst_items[j-1][0] or lst_items[j][8] != lst_items[j-1][8] + 1:
                        break
                    # find child
                    for k in range(len(lst_items)-1,-1,-1):
                        if int(lst_items[k][1]) == int(lst_items[j][1]) + 1\
                            and k < j and lst_items[k][0] == n:
                            child_lst.append(lst_items[k])
                            break                
    
    #                 find same parent and child of the same parent of other request
                    for l in range(len(lst_items)):
                        if lst_items[l][2] == lst_items[j][2] and l != j and lst_items[l][7] != 'done':
                            lst_items[l][7] = 'done'
    
                            for m in range(len(lst_items)-1,-1,-1):   
                                if int(lst_items[m][1]) == int(lst_items[l][1]) + 1\
                                and m < l and lst_items[m][0] == lst_items[l][0]:
                                    if not any(lst_items[m][2] in child for child in child_lst): #lst_items[m] not in child_lst:
                                        child_lst.append(lst_items[m])
                                    break
    
                    # find same childrent level and collect parent, try if child = 1
                    if len(child_lst) == 1:
                        par_lst = find_par(child_lst, par_lst, lst_items)
                        for k in range(len(lst_items)):
                            if lst_items[k][2] == child_lst[0][2] and k != child_lst[0][8]:
                                child_lst1 = []
                                child_lst1.append(lst_items[k])
                                par_lst = find_par(child_lst1, par_lst, lst_items)
                    # print('1-1111 n and num and letter', n_increase, num, letter)
                    increase = True
                    out_txt , num, letter = txt_list(par_lst,child_lst,out_txt, num, letter,n_increase)
                    # print('1-1111 n and num and letter return', n_increase, num, letter)
        if increase:
            # print('1-1112 increase at n',n_increase,n)
            n_increase +=1
    return out_txt
# write_script()

In [190]:
# write_script()

## Ask open script

In [191]:
def ask_open_script():

    ask_open_script = tk.messagebox.askyesno(title='Completed',
        message='Script done, do you want to open?') ##, parent = top_box)
    if ask_open_script:
        programName = "notepad.exe"
        fileName = "file.txt"
        sp.Popen([programName, script_file])
        

## Create popup message

In [192]:
### Create message
def close_popup(popup):
    popup.destroy()

def create_popup(title, mes, time):
    popup = tk.Toplevel(window)
    popup.attributes('-topmost',True)
    popup.title(title)
    popup.geometry("300x150")
    popup_label = tk.Label(popup, text = mes)
    popup_label.pack(pady=20)
    popup.after(time, lambda: close_popup(popup))
    

## Button Write Script

In [193]:
## write script
def script_btn():
    extract_path()
    if len(file_full) < 1:
        return
    read_excel(file_full)
    print('request after read_excel() ', request)
    get_cur_list()
    if len(cur_pn_list) < 1:
        return
    print('check status download WUBs: ',var_check_down.get())
    if var_check_down.get() == 1:
        print('check box download WUBs')
        get_main_page()
        print('after run goto_home', main_page)
        error = down_wub()
        if error == True:
            return
    get_project_key()
    write_script()
    # try: write_script()
    # except: 
    #     tk.messagebox.showinfo(title='Write Script Error',
    #         message='Error in write script, check and close relative files')
    #     return
    ask_open_script()

# ECO Frame

## def get_script_file()

In [194]:
### get form file
def get_script_file():
    script_file_path = filedialog.askopenfilename(title= "Please select Script file",\
                filetypes=[("Text Document", ".txt")])
    print('len of Script request form path: ',len(script_file_path))
    if len(script_file_path) > 3:
        entry_script_file.delete(0,END)
        entry_script_file.insert(0,script_file_path)
    if len(entry_script_file.get()) > 3:
        script_file = entry_script_file.get()
    

    

## get ECO page

In [195]:
### get ECO page
def get_eco_page():
    eco_num = entry_ECO_num.get().strip()
#     main_page = driver.current_window_handle
    sleep(1)
    driver.switch_to.window(main_page)
#     driver.maximize_window()
    try:
        btn_plus = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                        '//*[@id="tvc-bin-icon"]/i')))
        sleep(random.randint(2, 3))
    except TimeoutException:
        driver.refresh()
        sleep(random.randint(5, 7))
        try:
            print('try 2nd time to get Enovia site after refresh')
            btn_plus = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                            '//*[@id="tvc-bin-icon"]/i')))
            sleep(random.randint(2, 3))
        except:
            tk.messagebox.showinfo(title='Error 101', \
            message='Can not load Enovia site, check your connection and try again')
            return
            # sys.exit()
        sleep(random.randint(2, 3))
    #     if len(driver.window_handles) > 1:
    #             print(main_page)
    #         for windown in driver.window_handles:
    #             if windown != main_page:
    #                 driver.switch_to.window(windown)
    #                 driver.close()
    driver.switch_to.window(main_page)
    search_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
                                                        '//*[@id="quickSearch"]')))
    search_field.send_keys(eco_num)
    #     sleep(random.randint(0, 1))
    search_field.send_keys(Keys.RETURN)
    driver.switch_to.window(driver.window_handles[-1])
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    #     driver.switch_to.frame('tableContentFrame')
    driver.switch_to.frame('tableBodyRight')
    ecos = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    if len(ecos) == 1:
    #     print('one ECO found')
        eco_select = driver.find_element(By.XPATH, '//*[@id="tbr"]/tbody/tr/td/a')
    else:
        print('no or more than one ECO found')
        eco_select_number = simpledialog.askstring(title="Script to Enovia",
                                      prompt="Please select ECO, input 1, 2 or 3:")
        eco_select = driver.find_element(By.XPATH,\
                    '//*[@id="tbr"]/tbody/tr[' + str(eco_select_number) + ']/td/a')
    eco_select.click()
    sleep(5)
    driver.switch_to.window(driver.window_handles[-1])
    # eco_win_return = driver.current_window_handle
#     driver.switch_to.window(eco_win)
# #     driver.maximize_window()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    affected_bar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                                    'All Affected Items')))
    affected_bar.click()
    sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    eco_win_return = driver.current_window_handle
#     print(eco_win)
    return eco_win_return

## def go to ECO button 

In [196]:
### go to Enovia
def goto_ECO_btn():
    global ECO_win
    print('ECO Number: ',entry_ECO_num.get())
    get_main_page()
    # try:
    #     # print('try switch to home page')
    #     driver.switch_to.window(main_page)
    #     # print('switched to home page')
    # except:
    #     # print('main page is not existed')
    #     main_page = get_home_page()
        # print('get home page after first initial')
    try:
        print('try switch to ECO page')
        driver.switch_to.window(ECO_win)
        print('switched to ECO page')
    except:
        print('ECO page is not existed')
        ECO_win = get_eco_page()
        print('get ECO page after first initial')

# Executive Option frame

## part_pattern

In [197]:
part_pattern = \
re.compile(r'\b\d+[A-Z]*\d+\d+[-]*\d*[A-Z]*\d+\d+[-]*\d+[-_ ]{0,1}[A-Z]{0,2}[\W_]{1}') #[\b]*')
dash_pattern = re.compile(r'[oO]\W+[_]?\d\d\d?[\W?]') #re.compile(r'o\W+[_]?\d{3}[\W?]')
rev_pattern = re.compile(r'[oO]\W*?[-_ ][A-Z][A-Z]\b')#(r'o\W+[_-]?[A-Z][A-Z][\W?]')
des_pattern = re.compile(r'[\(\[].{5,45}[\)\]]')
qty_change = re.compile(r'[oO]\W+\d{0,5}[,.]?\d*',re.IGNORECASE)
qty_add = re.compile(r'\W+\d{1,5}[,.]?\d*',re.IGNORECASE)
sub_pur_roll = r'[\W+a-z_O]'
sub_pur_pn = r'[\W+a-z]'

## def pure_name(matched):

In [198]:
# function to filter dash roll
def pure_name(matched): # for part number find
    if matched.group(0) == '-':
        return '-'
    elif matched.group(0) == ' ':
        return ' '
    else:
        return ''

def pure_qty(matched): # for part number find
    if matched.group(0) == ',' or matched.group(0) == '.':
        return '.'
    else:
        return ''
special_char_list = ['_','-','(',')','.', ' ']
def pure_pn(pn):
    
    if any(pn.strip().split()[0][-1] in special_char for special_char in special_char_list):
        # print(pn[:-1]) .strip().split()[0]
        # print(pn.strip().split()[0][-1])
        return pn.strip().split()[0][:-1].strip()
        
    else:
        # print(pn, 'need to remove end')
        return pn.strip().split()[0]

## def extract_def

In [199]:
# extract and get data from a line then return [[pn,procedure,tfw,header,par_child], new description, ...]
def extract_def(line):
# for line in lines:
    cannot_find = []
    return_list = []
    
    if len(line) < 15:
        return []
    raw_parts = part_pattern.findall(line)

    pn = ''
    procedure = ''
    tfw = ''
    header = ''
    par_child = ''
    des_raw_result = ''
#     print(line)
#     print(raw_parts,raw_dash,raw_rev)
    if line.strip()[0].isdigit():
#         print(line[0], 'isdigit',line)
        header = 'header'
    if 'below' in line.lower() and 'bom' in line.lower():
        par_child = 'par'
    elif header == 'header' and ('reflect' in line.lower() or 'capture' in line.lower()):
        par_child = 'child'
    else:
        par_child = ''
        
    # combine line from 1 part 
    if (len(raw_parts) > 0): # and ('replace' not in line.lower()): #\
        for i in range(len(raw_parts)):
            pn_i1 = ''
            pn = pure_pn(re.sub(sub_pur_pn,pure_name,raw_parts[i])) #[:11]
            
            if i == len(raw_parts) - 1:
                end_find = len(line)
            else:
                pn_i1 = pure_pn(re.sub(sub_pur_pn,pure_name,raw_parts[i+1])) #[:-3] #[:11]
                end_find = line.find(pn_i1)
            if i == 0:
                start_find = 1
            else:
                start_find = line.find(pn)  + len(pn) # + 3
            raw_dash = dash_pattern.findall(line[start_find:end_find])
            raw_rev = rev_pattern.findall(line[start_find:end_find])
            des_raw = des_pattern.findall(line[start_find:end_find])
            # print('raw_dash/rev/des: ',raw_dash,raw_rev,des_raw)
            if len(des_raw) > 0:
                des_raw_result = des_raw[-1][1:-1]
            else:
                des_raw_result = ''
            ### find procedure and to-for-with
            if 'dash' in line[start_find:end_find].lower() and 'roll' in line[start_find:end_find].lower():
                procedure = 'dash'
                if len(raw_dash) > 0:
                    dash = re.sub(sub_pur_roll,'',raw_dash[0])
                    if pn.split('-')[-1].isnumeric() and int(dash) < int(pn.split('-')[-1]) +1:
                        dash = int(pn.split('-')[-1]) + 1
                        tfw = pn[:-len(str(dash))] + str(dash)
                    else:
                        tfw = pn[:-len(str(int(dash)))] + str(int(dash))
                ### check if: 1234567-000 is dash rolled to 1234567-001 
                elif pn[:-3] == pn_i1[:-3]:
                    print(i,pn, ' in line format: ',line)
                    tfw = pn_i1
                    return_list.append([pn,procedure,tfw,header,par_child,des_raw_result])
                    break
                else:
                    dash = pn.split('-')[-1]
                    # print(pn, 'dash: ',dash)
                    if dash.isnumeric() and '-' in pn:
                        dash = int(dash) + 1
                        tfw = pn[:-len(str(dash))] + str(dash)
                    else:
                        tfw = pn + '-XXX'
            elif 'rev' in line[start_find:end_find].lower() and 'roll' in line[start_find:end_find].lower():
                procedure = 'rev'
                if len(raw_rev) < 1:
                    tfw = 'no_rev' #re.sub(sub_pur_roll,'','not found rev')
                else:
                    tfw = re.sub(sub_pur_roll,'',raw_rev[0])

            elif 'release' in line.lower() and not ('pre-release' in line.lower()\
                or 'pre release' in line.lower() or 'prerelease' in line.lower()\
                        or 'pre_release' in line.lower() or 'pre/release' in line.lower()):
#             print(line,'release')
                procedure = 'release'
                tfw = ''
            elif 'inactive' in line.lower():
    #             print(line,'inactive')
                procedure = 'inactive'
                tfw = ''
            elif 'replace' in line.lower() and len(raw_parts) == 2:
                procedure = 'replace'
                tfw =  re.sub('[\W+a-z]',pure_name,raw_parts[1])
                des_raw = des_pattern.findall(line[end_find:len(line)])
                if len(des_raw) > 0:
                    des_raw_result = des_raw[-1][1:-1]
                else:
                    des_raw_result = ''
                return_list.append([pn,procedure,tfw,header,par_child,des_raw_result])
                break
            elif 'remove' in line.lower():
                procedure = 'remove'
            elif 'add' in line.lower():
                procedure = 'add'
                qty_letter = re.search(r'qty',line,re.IGNORECASE)
                if qty_letter is not None:
                    start_f = qty_letter.end()
                    qty_num = qty_add.findall(line[start_f:])
                    if len(qty_num) == 1:
                        tfw = re.sub(sub_pur_roll,pure_qty,qty_num[0])
                    else:
                        print('cannot found qty to add', line)
                        tfw = ''
                else:
                    tfw = ''
            elif ('change' in line.lower()) and ('qty' in line.lower() or 'quantity' in line.lower()):
                procedure = 'change'
                qty_letter = re.search(r'qty',line,re.IGNORECASE)
                if qty_letter is not None:
                    start_f = qty_letter.end()
                    qty_num = qty_change.findall(line[start_f:])
                if len(qty_num) == 1:
                    tfw = re.sub(sub_pur_roll,pure_qty,qty_num[0])
                else:
                    print('cannot found qty to change',line)
                    tfw = ''
            elif len(raw_parts) == 1 and not (\
                ('change' in line.lower())\
                or ('release' in line.lower())\
                or ('inactive' in line.lower())\
                or ('remove' in line.lower())\
                or ('add' in line.lower())\
                or ('roll' in line.lower())\
                                             ):
                pn = re.sub('[\W+a-z]',pure_name,raw_parts[0])[:11]
            return_list.append([pn,procedure,tfw,header,par_child,des_raw_result])
    elif len(raw_parts) < 1:
        if 'release' in line.lower():# and not ('pre-release' in line.lower()\
            # or 'pre release' in line.lower() or 'prerelease' in line.lower()\
            #             or 'pre_release' in line.lower() or 'pre/release' in line.lower()) :
            procedure = 'release'
            return_list.append([pn,procedure,tfw,header,par_child,des_raw_result])
        elif 'inactive' in line.lower():
            procedure = 'inactive'
            return_list.append([pn,procedure,tfw,header,par_child,des_raw_result])
        else:
            print('can not extract for line:',line)
            return_list.append([pn,procedure,tfw,header,par_child,des_raw_result])

    return return_list


## def collect_boms(lines)

In [200]:
### collect_BOMs
def collect_boms(lines):
    i = 0
    boms = []
    while i < len(lines):
    #     print(i)
        i += 1
        extracts = extract_def(lines[i-1])
        # print('extracts: ',extracts)
        if len(extracts) < 1:
            continue
        
        if extracts[0][3] == 'header':
            print('extracts: ',extracts)
            child_ext = []
            par_ext = []
            if extracts[0][4] == 'child':
                j = i - 1
                while j < len(lines) - 1:
                    j += 1
                    if lines[j][0].isdigit():
                        break
                    if len(extract_def(lines[j])) < 1:
                        continue
                    par_ext = extract_def(lines[j])
                    child_ext = extracts
                    boms.append(par_ext + child_ext)
                i = j
            elif extracts[0][4] == 'par':
                print('par',extracts)
                for extract in extracts:
                    par_ext = [extract]
                    j = i - 1
                    while j < len(lines) - 1:
                        j += 1
                        if lines[j][0].isdigit():
                            break
                        if len(extract_def(lines[j])) < 1:
                            continue
                        for extract in extract_def(lines[j]):
                            child_ext.append(extract)
                    boms.append(par_ext + child_ext)
                i = j
    #         elif extracts[0][1] == 'release' and extracts[0][0] == '':
    #             j = i - 1
    #             while j < len(lines) - 1:
    #                 j += 1
    #                 if lines[j][0].isdigit():
    #                     break
    #                 if len(extract_def(lines[j])) < 1:
    #                     continue
    #                 if not any(extract_def(lines[j])[0][0] in add for add in add_release):
    #                     add_release.append(extract_def(lines[j])[0])
    # #                     print('release',extract,extract[0])
    #             i = j
    #         elif extracts[0][1] == 'inactive' and extracts[0][0] == '':
    #             j = i - 1
    #             while j < len(lines) - 1:
    #                 j += 1
    #                 if lines[j][0].isdigit():
    #                     break
    #                 if len(extract_def(lines[j])) < 1:
    #                     continue
    #                 if not any(extract_def(lines[j])[0][0] in add for add in add_inactive):
    #                     add_inactive.append(extract_def(lines[j])[0])
    #             i = j
            # else:
            #     ask = tk.messagebox.showinfo('Info.','Line head but can not collect BOM: ' + str(extracts[0]))
    
    return boms

## def read_script(script_file_def)

In [201]:
### read txt file
def read_txt(script_file_path):
    with open(script_file_path) as f:
        lines = f.readlines()
    return lines

## def read_BOMs(lines):

In [202]:
### Read BOM
def read_BOMs():
    global boms, lines
    script_file = entry_script_file.get().strip()
    lines = read_txt(script_file)
    boms = collect_boms(lines)
    msg_boms = []
    for number, bom in enumerate(boms):
        msg_boms.append('_____________________________________________________________')
        msg_boms.append('Bom #' + str(number + 1) + ' Parent is:')
        print('Bom #',number + 1,'Parent is:')
        msg_boms.append(bom[0])
        print(bom[0])
        
        msg_boms.append('Below is childen:')
        print('Below is childen:')
        for i in range(1,len(bom)):
            msg_boms.append(bom[i])
            print(bom[i])
        
        print('')
    return msg_boms

## def shown_BOMs_btn(title):

In [203]:
### Create table to display BOM
def shown_BOMs_btn(title):
    # global boms, lines
    
    if len(entry_script_file.get()) < 3:
        get_script_file()
    # else:
    if len(entry_script_file.get()) < 3:
        return
    script_file = entry_script_file.get().strip()
    
    # lines = read_txt(script_file)
    # print('lines: ',lines)
    # boms = read_BOMs(script_file)
    popup = tk.Tk()
    # popup.attributes('-topmost',True)
    popup.title(title)
    popup.columnconfigure(0, weight=1) 
    popup.rowconfigure(0, weight=1)

    txt = Text(popup)
    txt.grid(row=0, column=0, sticky="eswn")
    scroll_y = Scrollbar(popup, orient="vertical", command=txt.yview)
    scroll_y.grid(row=0, column=1, sticky="ns")
    # bind txt to 
    txt.configure(yscrollcommand=scroll_y.set)
    
    BOMs_list = "\n".join([str(line).replace("'","").replace("[","").replace("]","")\
                    for line in read_BOMs()])
    
    txt.insert("1.0", BOMs_list)
    # make the text look like a label
    txt.configure(state="disabled", relief="flat", bg=popup.cget("bg"))
# shown_BOMs_btn(title)

## def collect add list

In [204]:
def collect_add_list(lines):
    global add_dash,add_rev,add_release,add_inactive,add_check_plant
    add_dash = []
    add_rev = []
    add_release = []
    add_inactive = []
    add_check_plant = []
    for line in lines:
        extracts = extract_def(line)
        if len(extracts) < 1:
            continue
            
        for extract in extracts:
#             print('extract',extract)
            if extract[0] != '':
                if extract[1] == 'dash':
                    if not any(extract[0] in add for add in add_dash):
    #                     print('dash',extract,extract[0])
                        add_dash.append(extract)
                elif extract[1] == 'rev':
                    if not any(extract[0] in add for add in add_rev):
                        add_rev.append(extract)
    #                     print('rev',extract,extract[0])
                elif extract[1] == 'release':
                    if not any(extract[0] in add for add in add_release):
                        add_release.append(extract)
    #                     print('release',extract,extract[0])
                elif extract[1] == 'inactive':
                    if not any(extract[0] in add for add in add_inactive):
                        add_inactive.append(extract)
                elif extract[1] == 'add':
                    if not any(extract[0] in add for add in add_check_plant):
                        add_check_plant.append(extract)


## collect search key

In [205]:
# key for add for dash or rev rolled
def collect_search_key():
    global search_key
    search_key = []
    for part in add_dash:
        print(part[0],'part[0] dash',part[0][:-len(part[0].split('-')[-1]) -1] )
        if not any(part[0][:-len(part[0].split('-')[-1]) -1] in key for key in search_key):
            search_key.append(part[0][:-len(part[0].split('-')[-1]) -1] +'*,')
    #         print(search_key)
    for part in add_rev:
        print(part[0],'part[0] rev',part[0][:-len(part[0].split('-')[-1]) -1] )
        if not any(part[0][:-len(part[0].split('-')[-1]) -1] in key for key in search_key):
            search_key.append(part[0][:-len(part[0].split('-')[-1]) - 1] +'*,')
    print('key search for affected items:')
    print(*search_key,sep='\n')
    return search_key

## refresh page

In [206]:
# refresh page
def refr_pg(page_to_refresh):
    print('refresh page ...')
    driver.switch_to.window(page_to_refresh)
    try:
        driver.switch_to.default_content()
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
        '/html/body/div[3]/div/div/div[2]/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/span/table/tbody/tr/td[4]/div/a/button')))
        refresh.click()
        print('refreshed')
    
    except:
        try:
            driver.switch_to.default_content()
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
            driver.switch_to.frame('detailsDisplay')
            driver.switch_to.frame('tableContentFrame')
            refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
                 '//*[@id="toolbar-container"]/div/div/div[4]/i')))
            refresh.click()
            print('refreshed')
        except:
            return

    sleep(1)

## def add_items():

In [207]:
def add_items():
# check if part added to affected items
    driver.switch_to.window(ECO_win)
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    affected_items_bar = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                            '//*[@id="tvcTabs0"]/div[1]/ul/li[2]/div/a[1]')))
    affected_items_bar.click()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    print('check if part added to affected items')
    for i in range(len(search_key)-1,-1,-1):
        for item in affected_items:
            if search_key[i][:7] in item.text:
                print(search_key[i],'has been added to affected items')
                search_key.pop(i)
                break
    
    for i in range(len(add_release)-1,-1,-1):
        for item in affected_items:
            if add_release[i][0][:7] in item.text:
                print(add_release[i],'has been added to release')
                add_release.pop(i)
                break
    
    if len(search_key) > 0:
        print('search_key remain',search_key)

        # need to deal with more than 150 items
        driver.switch_to.window(ECO_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tableContentFrame')
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        actions_btn.click()
        rev_dash_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Add For Revise/Dashroll')))
        print('got Add For Revise/Dashroll')
        sleep(random.randint(0, 2))
        rev_dash_btn.click()

        # input search key
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableContentFrame')
        names_input = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/form/table/tbody/tr[3]/td[2]/table/tbody/tr/td[1]/textarea')))
        names_input.clear()
        names_input.send_keys(search_key)
        print('key has input')
        sleep(random.randint(0, 2))
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableBottomFrame')
        find_affected_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                        '/html/body/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]')))
        print('find_affected_btn found')
        sleep(random.randint(0, 2))
        find_affected_btn.click()

        # pagination
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableBottomFrame')

        pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                '//*[@id="tvcPageControlOff"]')))
        pagination.click()

        print('select correct items')
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableContentFrame')
        driver.switch_to.frame('tableBodyRight')
        rerult_list = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
        print('found total',len(rerult_list),'items')

        s = 0
    #     add_release = []
    #     add_rev_dash = []

        for number, part in enumerate(add_dash):
            break_part = False   

            # find add for release dash roll
            for i in range(len(rerult_list)):
                if part[2] in rerult_list[i].text:# and 'preliminary' in rerult_list[i].text.lower():
                    add_dash.pop(number)
                    if not any(part[2] in item for item in add_release):
                        add_release.append(part)
                        print(part, 'added for release')
                    break_part = True
                    break
            if break_part:
                continue
            for i in range(len(rerult_list)):
                if part[0] in rerult_list[i].text:
    #                 add_rev_dash.append(part)
                    s += 1
                    part_found = driver.find_element(By.XPATH,\
                            '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                    actions.move_to_element(part_found).perform()
                    part_found.click()
                    print(part,'selected')                
        for number, part in enumerate(add_rev):
            break_part = False 
            for i in range(len(rerult_list)):
                if part[0] in rerult_list[i].text\
                and part[2] in (rerult_list[i].text[rerult_list[i].text.index(part[0])+11\
                                    :rerult_list[i].text.index(part[0])+15]).strip():
    #                 print(rerult_list[i].text[rerult_list[i].text.index(part[0])+11\
    #                                 :rerult_list[i].text.index(part[0])+15])
                    add_rev.pop(number)

                    if not any(part[0] in item for item in add_release):
                        print(part, 'added for release')
                        add_release.append(part)
                    break_part = True
                    break
            if break_part:
                continue
    #                 # found curent rev
            new_rev = part[2]
            letters = string.ascii_uppercase
            if new_rev[-1] == 'A' and new_rev[0] != 'A':
                if new_rev[0] == 'J' or new_rev[0] == 'P' or new_rev[0] == 'R'\
                          or new_rev[0] == 'T' or new_rev[0] == 'Y':
                    index = letters.index(new_rev[0])
                    cur_rev = letters[index - 2] + 'Y'   
                else:
                    index = letters.index(new_rev[0])
                    cur_rev = letters[index - 1] + 'Y'   
            elif new_rev[0] == 'J' or new_rev[0] == 'P' or new_rev[0] == 'R'\
                          or new_rev[0] == 'T' or new_rev[0] == 'Y':
                index = letters.index(new_rev[-1])
                cur_rev = new_rev[0] + letters[index - 2] 
            else:
                index = letters.index(new_rev[-1])
                cur_rev = new_rev[0] + letters[index - 1] 

            # found correct part to up rev
            for i in range(len(rerult_list)):
                if part[0] in rerult_list[i].text\
                and cur_rev in rerult_list[i].text[rerult_list[i].text.index(part[0])+11:rerult_list[i].text.index(part[0])+15]:
                    s += 1
    #                 add_rev_dash.append(part)
                    part_found = driver.find_element(By.XPATH,\
                            '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                    actions.move_to_element(part_found).perform()
                    part_found.click()
                    print(part, 'selected')
        if s > 0:
            ask = tk.messagebox.askokcancel('Confirm',\
                'Please check: ' + str(s) + ' part(s) selected to add to Affected items. Click Select, then click Ok to continue or Cancel to quit')
            if ask == False:

                sys.exit()
        #     print('selected total:',s,'items')
            print('below part need to add for release:')
            print(*add_release, sep = '\n')

    # go to add for release items, if len(add_release) > 0:
    if len(add_release) > 0:
        print(str(len(add_release)) + ' parts need to add for release')
#     else:
        driver.switch_to.window(ECO_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        print('actions button found')
        actions_btn.click()
        print('actions button click')
        rev_dash_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Add For Release')))
        print('got Add For Release')
        sleep(random.randint(0, 2))
        rev_dash_btn.click()

        # input search key
    #     driver.switch_to.window(driver.window_handles[2])
        driver.switch_to.default_content()
        driver.switch_to.frame('content')
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tvcInlineFrame_-1')
        driver.switch_to.frame('searchContentFrame')
        names_input1 = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/div[1]/div/div[3]/form/table/tbody/tr[6]/td/table/tbody/tr/td[1]/textarea')))
        print('names_input found')
        names_input1.clear()
        for part in add_release:
            names_input1.send_keys(part[0][:7] + '*\n')
        # names_input.send_keys(search_key)
        # print('names_input has input')
        submit_btn1 = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                        '//*[@id="SUBMIT_BUTTONS"]/a[2]')))
        submit_btn1.click()

        #pagination
        driver.switch_to.window(driver.window_handles[2])
        driver.switch_to.default_content()
        driver.switch_to.frame('content')
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tvcInlineFrame_-1')
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableBottomFrame')

        pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                '//*[@id="tvcPageControlOff"]')))
        pagination.click()

        # select part for release
    #     driver.switch_to.window(driver.window_handles[2])
        driver.switch_to.default_content()
        driver.switch_to.frame('content')
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tvcInlineFrame_-1')
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableContentFrame')
        driver.switch_to.frame('tableBodyRight')
        items_found = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')

        # if len(add_release) > 0:
        pn_release_selected = 0
        for part in add_release:
            if part[1] == 'dash':
                part_find = part[2]
            else:
                part_find = part[0]
            print('finding part:',part)
            for i in range(len(items_found)):
                if part_find in items_found[i].text:

                    try:
                        select_part = driver.find_element(By.XPATH,\
                                '/html/body/form/div[2]/div/table/tbody/tr[' + str(i+1) + ']/td/table/tbody/tr/td[2]/input')
                        actions.move_to_element(select_part).perform()
                        select_part.click()
                        print(i,part_find,'found --- selected',items_found[i].text)
                        pn_release_selected += 1
                    except:
                        print('part found but can not selected')
        if pn_release_selected > 0:
            ask = tk.messagebox.askokcancel('Confirm','Please check, Submit parts add for release, then click Ok to continue or Cancel to quit')
            if ask == False:
                print('exit program')
                sys.exit()

    # switch to affected items window
    driver.switch_to.window(ECO_win)
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))

    driver.switch_to.frame('detailsDisplay')
    driver.switch_to.frame('tableBottomFrame')
    pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                '//*[@id="tvcPageControlOff"]')))
    pagination.click()

    # refresh page
    print('try to refresh page ...')
    refr_pg(ECO_win)
    
    # work with affected window
    driver.switch_to.window(ECO_win)
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    # sleep(2)
    
    # add for dash rolled first
    s = 0
    for part in add_dash:
    #     print('part',part)
        for i in range(len(affected_items)):
            cur_p_dash = driver.find_element(By.XPATH,\
            '/html/body/form/div[3]/div[2]/table/tbody/tr[' + str(i+1) + ']/td[3]/a')
            uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[13]')
            new_p_dash = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[18]')

            if uom.text != '' and cur_p_dash.text == part[0] and new_p_dash.text == '':
                print('cur_p_dash',cur_p_dash.text)
                print('uom',uom.text)
                print('new_p_dash',new_p_dash.text)

                part_dash = driver.find_element(By.XPATH,\
                        '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                actions.move_to_element(part_dash).perform()
                part_dash.click()
                s += 1


    # up dash roll selected
    if s > 0:
        driver.switch_to.window(ECO_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tableContentFrame')
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        print('actions button found')
        actions_btn.click()
        print('actions button click')
        dash_selected = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Dashroll Selected')))
        print('Dashroll Selected')
        sleep(random.randint(0, 2))
        dash_selected.click()
        ask = tk.messagebox.askokcancel('Confirm','Please select Dashroll Selected, confirm, then click Ok to continue')
        if ask == False:
            # print('exit program')
            return
            # sys.exit()


    # find part for rev roll
    driver.switch_to.window(ECO_win)
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    sleep(2)
    
    # add for rev rolled rolled first
    r = 0
    for part in add_rev:

        for i in range(len(affected_items)):
            new_pn = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[18]')
            cur_pn = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[3]')
            uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[13]')

            if cur_pn.text == part[0] and new_pn.text == '' and uom.text != '':

                print(re.findall(part[0], affected_items[i].text),'has selected')
                part_rev = driver.find_element(By.XPATH,\
                        '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                actions.move_to_element(part_rev).perform()

                part_rev.click()
                r += 1
            elif new_pn.text == part[0] and uom.text != '':
                print(part[0],' has up rev roll')

    print('has',r,'need to up rev')

    # up rev roll selected, dont need to confirm
    if r > 0:
        driver.switch_to.window(ECO_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tableContentFrame')
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        print('actions button found')
        actions_btn.click()
        print('actions button click')
        rev_selected = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Revise Selected')))
        print('Revise Selected')
        sleep(random.randint(0, 2))
#     if r > 0:
        ask = tk.messagebox.askokcancel('Confirm','Please select rev rolled selected parts, then click Ok to continue or Cancel to quit')
        if ask == False:
            print('exit program')
            return

## def add affected items button 

In [208]:
def add_affected_items_btn():
    # global affected_win
    script_file = entry_script_file.get().strip()
    lines = read_txt(script_file)
    collect_add_list(lines)
    collect_search_key()


    ### get eco win with affected items bar
    try:
        print('try switch to ECO win')
        driver.switch_to.window(ECO_win)
    except:
        print('except get ECO win')
        goto_ECO_btn()
        try:
            print('try switch to ECO win')
            driver.switch_to.window(ECO_win)
        except:
            ask = tk.messagebox.askokcancel('Confirm','Can not get ECO-Affected window')
            return
    ### add affected items
    collect_search_key()
    add_items()
    thread = threading.Thread(target = create_popup, \
            args = ('Affected Items', 'Affected Items Adding: Done', 10000))
    thread.start()

## update BOM def:

In [209]:
# update BOMs
def update_boms(boms, ECO_win):
#     go to Affected bar
    driver.switch_to.window(ECO_win)
    sleep(2)

    first_assy = True
    for number, bom in enumerate(boms):
        print(f'go to udpate for BOM {number+1}/{len(boms)}: {bom[0]}')
        driver.switch_to.window(ECO_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
        affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')

        if bom[0][1] == 'dash':
            assy_find = bom[0][2]
            rev_find = 'AA'
        elif bom[0][1] == 'rev':
            assy_find = bom[0][0]
            rev_find = bom[0][2]
        else:
            print('can not find assy and rev to find, break at extract',bom[0])
            continue #pass #return
        for i in range(len(affected_items)):
            driver.switch_to.window(ECO_win)
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
            uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[28]')
            new_assy = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[18]')
            # just find key on name in new rev/doash rolled, do not check the dash rolled
            if uom.text != '' and new_assy.text.split('-')[0] == assy_find.split('-')[0] :
    #             first_assy = True
                open_in = driver.find_element(By.XPATH,\
                            '//*[@id="tbr"]/tbody/tr[' + str(i+1) + ']/td[16]/a/img')
                actions.move_to_element(open_in).perform()
                open_in.click()
                print('found assy ______:',assy_find,'item',i,'of',len(affected_items))
                sleep(2)
                driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
                bom_win = driver.current_window_handle      
#                 driver.maximize_window()
                if first_assy:
                    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                    ebom_btn = WebDriverWait(driver, 2).until\
                         (EC.presence_of_element_located((By.XPATH,\
                            '/html/body/div[3]/div/div/div[3]/div[1]/ul/li[2]/div/a[1]')))
                    actions.move_to_element(ebom_btn).perform()
                    ebom_btn.click()
                    first_assy = False
                    print('first_assy has change to:',first_assy)
                driver.switch_to.window(bom_win)
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                part_in_bom = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
                child_count = len(part_in_bom)
                print('count parts in BOM:',child_count)
                for j in range(1, len(bom)):

                    child = bom[j]
                    print('updating for child: ',child)
                    if child[1] == 'rev'\
                    or child[1]  == 'dash'\
                    or child[1] == 'replace':
                        if child[1] == 'dash':
                            part_rep = child[2]
                            rev_rep = 'AA'
                        elif child[1] == 'replace':
                            part_rep = child[2]
                            rev_rep = ''
                        else:
                            part_rep = child[0]
                            rev_rep = child[2]
                        # sleep(1)
                        driver.switch_to.window(bom_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            rev_child = driver.find_element(By.XPATH,\
                                    '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[3]')
    #                         print('PN and Rev:',pn_child.text,rev_child.text)
                            if len(rev_child.text) > 6 and rev_child.text[-6] == '.' \
                                and pn_child.text == part_rep:
                                break
                            elif (child[1] == 'dash' and child[2].split('-')[0] == pn_child.text.split('-')[0])\
                                and pn_child.text.split('-')[-1].isnumeric()\
                                and child[2].split('-')[-1].isnumeric()\
                                and (int(pn_child.text.split('-')[-1]) >= int(child[2].split('-')[-1])):
                                print('Child Dash rolled has been replaced',pn_child.text)
                                break
    
                            elif (child[1] == 'replace' and child[2].split('-')[0] == pn_child.text.split('-')[0])\
                                and pn_child.text.split('-')[-1].isnumeric()\
                                and child[2].split('-')[-1].isnumeric()\
                                and (int(pn_child.text.split('-')[-1]) >= int(child[2].split('-')[-1])):
                                print('Child Dash rolled or replace has been replaced',pn_child.text)
                                break
                    
                            elif child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('found child:',child)
                                select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                                select_all.click()
                                select_all.click()
                                try:
                                    select_part = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div/table/tbody/tr['\
                                                    + str(k+1) + ']/td[2]/input[1]')
                                    actions.move_to_element(select_part).perform()
                                    select_part.click()
                                except:
                                    print('part found but can not selected, break')
                                    break
                                driver.switch_to.window(bom_win)
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                                actions_btn = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '//*[@id="toolbar-container"]/div/div/div[4]/div[1]/span')))
                                actions.move_to_element(actions_btn).perform()
                                actions_btn.click()
                                try:
                                    rep_with_existing = WebDriverWait(driver, 2).until\
                                        (EC.presence_of_element_located((By.LINK_TEXT,\
                                                'Replace with Existing')))
                                    rep_with_existing.click()
                                except:
                                    print('can not select button rep_with_existing for',child)
                                    break
                                driver.switch_to.window(driver.window_handles\
                                            [len(driver.window_handles)-1])
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('searchContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                                name_input = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '/html/body/form/table/tbody/tr[2]/td[2]/table/tbody/tr/td[1]/textarea')))
                                name_input.send_keys(part_rep[:7] + '*')
                                driver.switch_to.window(driver.window_handles\
                                                        [len(driver.window_handles)-1])
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('searchContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBottomFrame'))
                                find_btn = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '/html/body/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]/a')))
                                actions.move_to_element(find_btn).perform()
                                find_btn.click()
                                # go to result of finding
                                driver.switch_to.window(driver.window_handles\
                                                        [len(driver.window_handles)-1])
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('searchContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                                parts_found = driver.find_elements(By.XPATH,\
                                                                    '//*[@id="tbr"]/tbody/tr')
                                # select last part in list, need to check
                                select_part = driver.find_element(By.XPATH,\
                                        '//*[@id="tbl"]/tbody/tr['\
                                                + str(len(parts_found)) + ']/td[1]/input')
                                actions.move_to_element(select_part).perform()
                                select_part.click()
                                print('part selected:',child[0])

                                ask = tk.messagebox.askokcancel('Confirm',\
                                'Please Submit Replace Part, then click Ok to continue or Cancel to quit')
                                if ask == False:
                                    print('exit program')
                                    driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                    driver.close()  
                                    driver.switch_to.window(driver.window_handles[\
                                                    len(driver.window_handles)-1])
                                    driver.close() 
                                    sys.exit()
                                driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                if driver.current_window_handle != ECO_win\
                                    and driver.current_window_handle != bom_win:
                                    print('window is not BOM-window, closing')
                                    driver.close()    

                                # break after replace child to move next child
                                break
                    elif child[1] == 'add': # change to elif after testing                    
                        driver.switch_to.window(bom_win)    
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        part_added = False
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            if child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('Part has been added ',pn_child.text)
                                part_added = True
                                break
                        if part_added:
                            continue
                        driver.switch_to.window(bom_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                        select_all.click()
                        select_all.click()
                        driver.switch_to.window(bom_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))



                        actions_btn = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.XPATH,\
                                        '//*[@id="toolbar-container"]/div/div/div[4]/div[1]/span')))
                        actions.move_to_element(actions_btn).perform()
                        actions_btn.click()
                        try:
                            add_existing = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.LINK_TEXT,\
                                        "Add Existing [Drag'n Drop]")))
                            add_existing.click()
                        except:
                            print('can not select btn add_existing for',child_extract[0][0])
                            continue
                        driver.switch_to.default_content()
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        driver.switch_to.frame('tvcInlineFrame_-1')
                        driver.switch_to.frame('searchContentFrame')
                        input_area = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.XPATH,\
                        '//*[@id="tvcSearchForm"]/table/tbody/tr[4]/td/table/tbody/tr/td[1]/textarea')))
                        input_area.send_keys(child[0][:7] + '*')
                        submit_btn = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.LINK_TEXT,\
                                        "Submit")))
                        submit_btn.click()
                        driver.switch_to.default_content()
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        driver.switch_to.frame('tvcInlineFrame_-1')
                        driver.switch_to.frame('searchContentFrame')
                        driver.switch_to.frame('tableContentFrame')
                        driver.switch_to.frame('tableBodyRight')
                        tick_box = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.XPATH,\
                            '/html/body/form/div[2]/div[1]/table/tbody/tr/td[1]/table/tbody/tr/td[2]/input[1]')))
                        tick_box.click()
                        ask = tk.messagebox.askokcancel('Confirm','Please check Added Part, then drag to BOM')
                        if ask == False:
                            print('exit program')
                            driver.switch_to.window(driver.window_handles[\
                                                len(driver.window_handles)-1])
                            driver.close()  
                            sys.exit()
                        print('found part to add: ',child)
                        continue
                    elif child[1] == 'remove':
                        driver.switch_to.window(bom_win)
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            if child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('found child to remove:',child[0])
                                select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                                select_all.click()
                                select_all.click()
                                print('select_all')
                                try:
                                    select_part = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div/table/tbody/tr['\
                                                    + str(k+1) + ']/td[2]/input[1]')
                                    actions.move_to_element(select_part).perform()
                                    select_part.click()
                                except:
                                    print('part found but can not selected')
                                    break
                                driver.switch_to.window(bom_win)
                                driver.switch_to.frame('content')
                                driver.switch_to.frame('detailsDisplay')
                                driver.switch_to.frame('tableContentFrame')
                                actions_btn = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '//*[@id="toolbar-container"]/div/div/div[4]/div[1]/span')))
                                actions.move_to_element(actions_btn).perform()
                                actions_btn.click()

                                # select replace with existing part
                                try:
                                    rep_with_existing = WebDriverWait(driver, 2).until\
                                        (EC.presence_of_element_located((By.LINK_TEXT,\
                                                'Remove Selected')))
                                    rep_with_existing.click()
                                except:
                                    print('can not select for Remove Selected',child)
                                    break
                                ask = tk.messagebox.askokcancel('Confirm',\
                                    'Please check Part to remove, then confirm and click Ok to continue or cancel to Quit')
                                if ask == False:
                                    print('exit program')
                                    driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                    driver.close()  
                                    sys.exit()

                                break

                    elif child[1] == 'change':
                        driver.switch_to.window(bom_win)
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            if child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('found child to change qty:',child[0])
                                select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                                select_all.click()
                                select_all.click()
                                print('select_all')

                                try:
                                    select_part = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div/table/tbody/tr['\
                                                    + str(k+1) + ']/td[2]/input[1]')
                                    actions.move_to_element(select_part).perform()
                                    select_part.click()

    #                                 /html/body/form/div[2]/div[2]/table/tbody/tr[31]/td[8]
                                    child_qty = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                                    + str(k+1) + ']/td[8]')

                                    print('current Qty', child_qty.text)
    #                                     print(type(child_qty.text))
                                    if child_qty.text[:len(child[2])] == child[2]:
                                        print('Qty has changed before, break to next')
                                        break
                                    actions.move_to_element(child_qty).perform()
                                    child_qty.click()
    #                                 '/html/body/form/div[2]/div[2]/table/tbody/tr['\
    #                                                 + str(k+1) + ']/td[8]/input'
                                except:
                                    print('part found but can not selected')
                                    break



                                ask = tk.messagebox.askokcancel('Confirm',\
                                    'Please input Qty '+child[2]+' for '\
                                        +pn_child.text+', then click Ok to continue or cancel to Quit')
                                if ask == False:
                                    print('exit program')
                                    driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                    driver.close()  
                                    sys.exit()
                                space_click = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                                    + str(k+1) + ']/td[9]')
                                actions.move_to_element(space_click).perform()
                                space_click.click()
                                break
                print('closing ... at:',j+1,'of', len(bom))
                sleep(1)
                driver.switch_to.window(bom_win)
                if driver.current_window_handle != ECO_win:
                    print('window is not affected_win, closing')
                    driver.close()    
                # break to next assy
#                 break

    print('done')

## Update BOM button

In [210]:
def update_BOM_btn():
    print('run update bom')
    try: boms
    except: read_BOMs()
    try: driver.switch_to.window(ECO_win)
    except: goto_ECO_btn()
    update_boms(boms, ECO_win)
    thread = threading.Thread(target = create_popup, \
            args = ('Update BOM', 'Update BOM: Done', 10000))
    thread.start()

# Check parameter Option

## Extract script to list

In [211]:
def get_list_from_txt(pn_list, part_detail, lines):
    i = -1
    
    # print('file_path: ',file_path)
    
    while i < len(lines) - 1:
        i += 1
        raw_parts = part_pattern.findall(lines[i])
        # print('i: ',i)
        for extract in extract_def(lines[i]):
            # print('line i: ',i)
            # print('_________',extract)
            if extract[0] != '' and extract[1] == 'dash': 
                # print('dash: ',extract)
                if not any(extract[2].split('-')[0] in pn for pn in pn_list):
                    pn_list.append(extract[2].split('-')[0])
                    part_detail.append([extract[2] + '_AA',extract[1],extract[5]])
                else:
                    print('dash pn has in pn_list: ',extract[2])
            elif extract[0] != '' and extract[1] == 'rev': 
                # print('rev: ',extract)
                if not any(extract[0].split('-')[0] in pn for pn in pn_list):
                    pn_list.append(extract[0].split('-')[0])
                    part_detail.append([extract[0] + '_' + extract[2],extract[1],extract[5]])
                else:
                    print('rev pn has in pn_list: ',extract[0])
                    
            elif extract[0] != '' and extract[1] == 'add': 
                # print('add: ',extract)
                if not any(extract[0].split('-')[0] in pn for pn in pn_list):
                    pn_list.append(extract[0].split('-')[0])
                    part_detail.append([extract[0],extract[1], extract[5]])
                else:
                    print('rev pn has in pn_list: ', extract[0])
                    
            elif  extract[0] != ''  and extract[1] == 'replace':
                # print('replace: ',extract)
                if not any(extract[2].split('-')[0] in pn for pn in pn_list):
                    pn_list.append(extract[2].split('-')[0])
                    part_detail.append([extract[2],extract[1],extract[5]])
                else:
                    print('replace pn has in pn_list: ',extract[2])
            elif extract[0] != '' and  extract[1] == 'release':
                # print('release: ',extract)
                if not any(extract[0].split('-')[0] in pn for pn in pn_list):
                    pn_list.append(extract[0].split('-')[0])
                    part_detail.append([extract[0],extract[1],extract[5]])
                else:
                    print('release pn has in pn_list: ',extract[0])
            elif 'release' in extract[1].lower() and extract[3] == 'header':
                # print('release ....... ',extract)
                j = i
                while j < len(lines) - 1:
                    j += 1
                    # print('extract_def(lines[j])[0][3]:',len(extract_def(lines[j])))
                    if (len(extract_def(lines[j])) > 0 and extract_def(lines[j])[0][3] == 'header'): 
                        #or len(extract_def(lines[j])) < 1:
                        print('break at: ', j)
                        break
                    elif len(extract_def(lines[j])) > 0 and extract_def(lines[j])[0] != '':
                        if not any(extract_def(lines[j])[0][0].split('-')[0] in pn for pn in pn_list):
                            pn_list.append(extract_def(lines[j])[0][0].split('-')[0])
                            part_detail.append([extract_def(lines[j])[0][0],'release',\
                                        extract_def(lines[j])[0][5]])
                        else:
                            print('children release list pn has in pn_list: ',extract_def(lines[j])[0][0])
                    # print('part release: ',extract_def(lines[j]))
                i = j - 1
    return pn_list, part_detail

## ext_script_to_excel

In [212]:
def ext_script_to_excel():
    global file_path_para, wb, ws, key_check, part_detail
    pn_list = []
    part_detail = []

    # try: 
    #     file_path_para #= os.path.dirname(entry_script_file.get().strip())
    #     print('try and catch',file_path_para)
    # except:
    try: file_path_para
    except:
    # if len(file_path_para) > 0:
    #     print('has path')
        if len(entry_script_file.get()) < 3:
            file_path_para = filedialog.askdirectory(title= "Please select Drawings folder")
            if file_path_para =='':
                print('file none, return')
                return
        else:
            # script_file = 
            file_path_para = os.path.dirname(entry_script_file.get().strip())

    file_parameter = False
    for file in os.listdir(file_path_para):
        if ('parameter' in file.lower())\
        and (file.lower().endswith(".xlsx")):
            file_parameter_name = os.path.join(file_path_para, file)
            file_parameter = True
            break
    if file_parameter == False:
        file_parameter_name = 'Parameters Check.xlsx'
    

    if file_parameter:
        msg_keep_list = tk.messagebox.askquestion('Download option',\
        'Found Parameter template file in selected folder, Do you want to keep current list in Excel file',\
        icon='question')
        if msg_keep_list == 'yes':
            collect_data = tk.messagebox.askquestion('Download option',\
            'Do you want to collect data from files in selected folder?',\
            icon='question')
            
    key_input = '' #simpledialog.askstring(title="Check in Drawing",
        # prompt="Please input Keys to check in Drawing, separate by ', / ':")
    if key_input is not None:
        newList = re.split(',|_|\.|/|\*', key_input)
        key_check = [s.strip() for s in newList if s != '']
        print('key_check',key_check)
    else:
        key_check = ''

    


    
    # part_pattern = re.compile(r'\b\d{7}[-_ ]\d{3}[-_ ][A-Z][A-Z]_?\b') #_?[^\w]')
    # part_script_pattern = re.compile(r'\b\d{7}[-_ ]\d{3}\b')
    wb = xw.Book(file_parameter_name)
    ws = wb.sheets.active
    lr = ws.range('B' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
    
    if file_parameter and msg_keep_list == 'yes':
        for i in range(1,lr):
            if ws.range(i+1,2).value != None and len(ws.range(i+1,2).value) > 10:
                print(len(ws.range(i+1,2).value),'ws.range(i+1,2).value',ws.range(i+1,2).value)
                pn_list.append(ws.range(i+1,2).value.strip().split('-')[0])
        i += 1
    else:
        ws.range('2:'+str(lr+100)).api.Delete(DeleteShiftDirection.xlShiftUp) 
        i = 1
        
    

    # with open(script_file) as f:
    if  file_parameter == False or msg_keep_list == 'no' or collect_data == 'yes':
        # found_txt = False
        for file in os.listdir(file_path_para):
            if (file.lower().endswith(".txt")):
                print('extract data from script file:',file)
                script_path = file_path_para + '/' + file
                lines = read_txt(script_path)
                pn_list, part_detail = get_list_from_txt(pn_list, part_detail, lines)
                
                


    # loop and collect from pdf file
    for file in os.listdir(file_path_para):
        if (file.lower().endswith(".pdf"))\
        and 'qc' not in file.lower()\
        and len(file) < 50\
        and file.strip()[0].isdigit():
            print('Collecting file name:',file)
            raw_parts = part_pattern.findall(file)
            if len(raw_parts) > 0 and not any(raw_parts[0].split('_')[0].split('-')[0] in pn for pn in pn_list):
                pn = pure_pn(re.sub(sub_pur_pn,pure_name,raw_parts[0]))
                print('pn: ',pn,'...')
                pn_list.append(pn.split('_')[0].split('-')[0])
                part_detail.append([pn,''])
    # # return pn_list, part_detail
    for part in part_detail:
        i += 1
        print(i,'part[0]: ',part[0])
        ws.range(i,2).value = part[0]
        # print('ws.range(i,2).value',ws.range(i,2).value)
        if part[2] != '':
            ws.range(i,3).value = part[2]
        else:
            ws.range(i,3).value = ''
        ws.range(i,1).value = i - 1
    
    # print(*pn_list, sep = '\n')
    # print(*part_detail,sep = '\n')
# print(pn_list)
# print(part_detail)

## Extract part number

In [213]:
# def drw_extract(file_path, file, pn_input,des_input, key_check):
# pn_input ='058-70-xxx AA'
    # red_line = False
rev_patt_pn = re.compile(r'\W*?[-_ ][A-Z][A-Z]\b')#(r'o\W+[_-]?[A-Z][A-Z][\W?]')

def ext_pn(pn_input):
    pn_input = pn_input.strip()
    pn = pn_input #[:11]
    pn_key = pn_input[:7]
    pn_dash = pn_input[8:11]
    # pn_rev = pn_input.split(' ')[-1].split('_')[-1]
    raw_rev = rev_patt_pn.findall(pn_input)
    
    # print('raw_rev: ',raw_rev, index)
    if len(raw_rev) > 0:
        print(raw_rev)
        index = pn.find(raw_rev[0])
        pn = pure_pn(re.sub(sub_pur_pn,pure_name,pn_input[:index]))
        # pn = pn_input[:index].strip()
        pn_rev = re.sub(sub_pur_roll,'',raw_rev[0])
        # pn_rev = raw_rev[0]
    else:
        pn = pn_input.split(' ')[0].split('_')[0]
        pn_rev = ''

    # print(pn.split('-'))
    if len(pn.split('-')) > 1:
        pn_split = pn.split('-') #.pop(0)
        pn_dash = pn_split[-1]
        del pn_split[-1]
        pn_key = '-'.join(pn_split)
        # print('pn_split: ',pn_split)
        # print('pn: ',pn)
    return pn, pn_key, pn_dash, pn_rev
# # print('pn: ',pn,'pn_key: ',pn_key,'pn_dash: ',pn_dash,'pn_rev:',pn_rev)
# print(ext_pn(' 0780606-000'))

## OMS extract

In [214]:
# check oms file
def oms_extract(file_path, file, pn_input,des_input, key_check):
    pn_result = []
    rev_result = []
    rev_table_result = ''
    des_result = ''
    key_check_result = []
    dash_x = ''
    
    red_line = False
    pn_drw = os.path.join(file_path, file)
    pn, pn_key, pn_dash, pn_rev = ext_pn(pn_input)
    # pn = pn_input[:11]
    # pn_key = pn_input[:7]
    # pn_dash = pn_input[8:11]
    # pn_rev = pn_input[12:14]
    revision_remove = re.compile(r'revision ?level',re.IGNORECASE)
    #     level_remove = re.compile(r'level',re.IGNORECASE)
    process_remove = re.compile(r'process *identification?:?',re.IGNORECASE)

    doc: Document = fitz.open(pn_drw)
    for i in range(len(doc)): #find key check in all page
        print('page',i+1,'______________________________________')
        page: Page = doc[i]
        title_block = [0,0,page.rect.width,100]

        pn_pos = [10,40,120,60]
        rev_pos = [page.rect.width-100,75,page.rect.width-10,100]
#         print('pn_pos',pn_pos,'rev_pos',rev_pos)
#         highlight = page.add_highlight_annot(pn_pos)
#         highlight.set_colors({"stroke":(0, 0.9, 0.9)})
#         highlight.update()
#         highlight = page.add_highlight_annot(rev_pos)
#         highlight.set_colors({"stroke":(0, 0.9, 0.9)})
#         highlight.update()
        
        des_pos_1 = [page.rect.width/2+50,60,page.rect.width/2+160,70]
        des_pos_2 = [page.rect.width/2-75,75,page.rect.width/2+160,85]
#         print('des_pos_1',des_pos_1,'des_pos_2',des_pos_2)
#         highlight = page.add_highlight_annot(des_pos_1)
#         highlight.set_colors({"stroke":(0, 0.5, 0)})
#         highlight.update()
#         highlight = page.add_highlight_annot(des_pos_2)
#         highlight.set_colors({"stroke":(0, 0.7, 0)})
#         highlight.update()
        
        doc_num_pos = page.search_for('document number',clip = title_block)
        assy_num_pos = page.search_for('assemb',clip = title_block)   
        rev_lvl_pos = page.search_for('revision level',clip = title_block)
        process_pos = page.search_for('process identification',clip = title_block)
        operation_pos = page.search_for('operation',clip = title_block)
        
        #find part number
#         print('doc_num_pos',len(doc_num_pos),doc_num_pos)
        if len(doc_num_pos) == 1:
            pn_pos[0] = doc_num_pos[0][0] - 5
            pn_pos[1] = doc_num_pos[0][3] + 2
            pn_pos[2] = doc_num_pos[0][2] + 5
            pn_pos[3] = pn_pos[1] + 15
        if len(assy_num_pos) > 0:
            pn_pos[3] = assy_num_pos[0][1] - 2
#         print('pn_pos after process_pos',pn_pos)
        pn_text = page.get_textbox(pn_pos)
        print('pn_text',pn_text)
        if pn_input[:7] in pn_text:
#             print('pn_input[:7] in pn_text')
            highlight = page.add_highlight_annot(pn_pos)
            highlight.set_colors({"stroke":(0, 0, 1)})
            highlight.update()
        else:
            red_line = True
            pn_result.append(f'Page {i+1}: ({pn_text})')
            highlight = page.add_highlight_annot(pn_pos)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()
        if 'x' not in pn_text.lower():
            print('Need to change dash to -XXX')
            dash_x = 'Need to change dash to -XXX'
            
        #find revision
        if len(rev_lvl_pos) > 0:
            rev_pos[0] = rev_lvl_pos[0][0] - 5
            rev_pos[1] = rev_lvl_pos[0][3] + 2
            rev_pos[2] = rev_lvl_pos[0][2] + 5
            rev_pos[3] =rev_pos[1] + 15
        
        
        rev_text = page.get_textbox(rev_pos)
        print('rev_text',rev_text)
        if pn_rev in rev_text:
#             print('pn rev in rev_text',rev_text)
            highlight = page.add_highlight_annot(rev_pos)
            highlight.set_colors({"stroke":(0, 0, 1)})
            highlight.update()
        else:
            red_line = True
            rev_result.append(f'Page {i+1}: ({rev_text})')
            highlight = page.add_highlight_annot(rev_pos)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()

        # find description
#         print('len process_pos',len(process_pos),process_pos)
        if len(process_pos) == 1:
            # des part 1 position
            des_pos_1[0] = process_pos[0][2] + 10
            des_pos_1[1] = process_pos[0][1] + 2      
#             print(operation_pos[0][0] > des_pos_1[0],operation_pos[0][0] ,des_pos_1[0])
            if len(operation_pos) == 1 and operation_pos[0][0] > des_pos_1[0]:
                des_pos_1[2] = operation_pos[0][0] - 15
            else:
                des_pos_1[2] = process_pos[0][2] + 120
            des_pos_1[3] = process_pos[0][3] - 2

            # Des part 2 position
            des_pos_2[0] = process_pos[0][0]
            des_pos_2[1] = process_pos[0][3] + 2
            des_pos_2[2] = des_pos_1[2]
            des_pos_2[3] = process_pos[0][3] + 20

#             print('des_pos_1 after process_pos',des_pos_1)
    #         highlight = page.add_highlight_annot(process_pos)
    #         highlight.set_colors({"stroke":(0, 0.9, 0)})
    #         highlight.update()

        else:
            print('can not find Document Number Position, make sure document following template')
        des_text_1 = page.get_textbox(des_pos_1).replace(':','')
#         print('des_text_1',des_text_1)
        des_text_2 = page.get_textbox(des_pos_2)
#         print('des_text_2',des_text_2)
        
        des_text = des_text_1 + des_text_2
        print('des_text',des_text)
        if len(des_text) > 40:
            red_line = True
            des_result = 'Des. len > 40 (' + des_text + ')'
            highlight = page.add_highlight_annot(des_pos_1)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()
        elif des_text == des_input:
            highlight = page.add_highlight_annot(des_pos_1)
            highlight.set_colors({"stroke":(0, 0, 0.8)})
            highlight.update()
            highlight = page.add_highlight_annot(des_pos_2)
            highlight.set_colors({"stroke":(0, 0, 0.8)})
            highlight.update()
            des_result = des_text
        else:
            red_line = True
            des_result = 'Des not match (' + des_text + ')'
            highlight = page.add_highlight_annot(des_pos_1)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()
            highlight = page.add_highlight_annot(des_pos_2)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()
        
        
        #find revision table and check 
        if pn_rev == '' and rev_text != '':
            pn_rev = rev_text            
        if i == 0:
            rev_table = [0,page.rect.height / 2 - 100,300,page.rect.height - 20]
    #         page.draw_rect(rev_table, width=1.5, color=(0, 1, 0))
#             highlight = page.add_highlight_annot(rev_table)
#             highlight.set_colors({"stroke":(0.95, 0.9, 0)})
#             highlight.update()
            temp_pos = page.search_for('0282646',clip = rev_table)
            des_chg_pos = page.search_for('description',clip = rev_table)
            rev_his_pos = page.search_for('history',clip = rev_table)
#             print('len(des_chg_pos)',len(des_chg_pos),des_chg_pos)
#             print('rev_his_pos',len(rev_his_pos),rev_his_pos)
            
            if len(temp_pos) == 1:
                rev_table[3] = temp_pos[0][1]
            if len(des_chg_pos) == 1:
                rev_table[1] = des_chg_pos[0][1]
                rev_table[2] = des_chg_pos[0][0]
            elif len(rev_his_pos) == 1:
                rev_table[1] = rev_his_pos[0][3] + 20
                rev_table[2] = rev_his_pos[0][0]
            
            
            rev_col_pos = page.search_for('rev',clip = rev_table)
#             print('rev_colume_pos',len(rev_col_pos),rev_col_pos)
            if len(rev_col_pos) == 1:
                rev_table[1] = rev_col_pos[0][3]
                rev_table[2] = rev_col_pos[0][2]
            rev_list = page.get_textbox(rev_table)
            print('rev_list',rev_list)
            
            if pn_rev not in rev_list:
                print('Revsion has not yet updated to Revision table')
                rev_table_result = 'Revsion has not yet updated to Revision table'
                highlight = page.add_highlight_annot(rev_table)
                highlight.set_colors({"stroke":(1, 0, 0)})
                highlight.update()                
        # check key in page
        if key_check != '':
            key_found = []
            for key in key_check:
                search_founds = page.search_for(key)
                if len(search_founds) > 0:
                    key_found.append(key)
                    red_line = True
                    for item in search_founds:
                        highlight = page.add_highlight_annot(item)
                        highlight.set_colors({"stroke":(1, 0, 0)})
                        highlight.update()
            if len(key_found) > 0:
                key_check_result.append(f'page {i+1}: {"/".join(key_found)}')
                
    if red_line:
        viz_name = os.path.join(file_path, file[:-4] + "________Redline.pdf" )
    else:
        viz_name = os.path.join(file_path, file[:-4] + "_SelfQC.pdf" )
    doc.save(viz_name)
    
    if len(pn_result) > 0:
        pn_result = 'PN not match in: ' + '; '.join(pn_result)
    else: pn_result = pn_text
    if dash_x != '':
        pn_result = pn_result + ' (' + dash_x +')'
    if len(rev_result) > 0:
        rev_result = 'Rev not match in: ' + '; '.join(rev_result)
    else: rev_result = rev_text
    if rev_table_result != '':
        rev_result = rev_result + ' (' + rev_table_result +')'
    if len(key_check_result) > 0:
        key_check_result = 'Found key in: ' + '; '.join(key_check_result)
    else: key_check_result = ''      
    
    return pn_result,rev_result,des_result,key_check_result

## Drawing Extract

In [215]:
def drw_extract(file_path, file, pn_input,des_input, key_check):
    red_line = False
    pn, pn_key, pn_dash, pn_rev = ext_pn(pn_input)
    # pn = pn_input[:11]
    # pn_key = pn_input[:7]
    # pn_dash = pn_input[8:11]
    # pn_rev = pn_input[12:14]

    pn_result = []
    rev_result = []
    rev_table_result = ''
    des_result = ''
    key_check_result = []
    pn_drw = os.path.join(file_path, file )
    doc: Document = fitz.open(pn_drw)

    for i in range(len(doc)): #find key check in all page
        page: Page = doc[i]
        text = page.get_text()
        if i == 0:
            title_block = [page.rect.width - 420,page.rect.height - 180,page.rect.width - 30,page.rect.height - 40]
            page.draw_rect(title_block, width=1.5, color=(0, 1, 0))
        else:
            title_block = [page.rect.width - 320,page.rect.height - 120,page.rect.width - 30,page.rect.height - 40]
            page.draw_rect(title_block, width=1.5, color=(0, 1, 0))

        pn_pos = []
        rev_roll_pos = []
        
        drw_num_pos = page.search_for('drawing number',clip = title_block)
        rev_pos = page.search_for('rev',clip = title_block)
        
        # find PN position
        if len(drw_num_pos) == 1:
            pn_pos.append(drw_num_pos[0][0])
            pn_pos.append(drw_num_pos[0][3]+1)
            if len(rev_pos) == 1:
                pn_pos.append(rev_pos[0][0]-10)
            else:
                pn_pos.append(drw_num_pos[0][2]+100)
            pn_pos.append(drw_num_pos[0][3] + 15)
        else:
            pn_pos.append(page.rect.width - 220)
            pn_pos.append(page.rect.height - 60)
            pn_pos.append(page.rect.width - 65)
            pn_pos.append(page.rect.height - 50)
            print('can not find Drawing number position',pn_pos)
        
        pn_text = page.get_textbox(pn_pos)
        print(i,'pn_text',pn_text)
        if pn_text == pn:
            page.draw_rect(pn_pos, width=1.5, color=(0, 0, 1))
        else:
            red_line = True
            pn_result.append(f'Page {i+1}: ({pn_text})')
            highlight = page.add_highlight_annot(pn_pos)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()
            
        # find rev position
        if len(rev_pos) == 1:
            rev_roll_pos.append(rev_pos[0][0]-5)
            rev_roll_pos.append(rev_pos[0][3]+2)
            rev_roll_pos.append(rev_pos[0][2]+5)
            rev_roll_pos.append(rev_pos[0][3] + 15)
        else:
            rev_roll_pos = [page.rect.width - 60,page.rect.height - 60,\
                           page.rect.width - 30,page.rect.height - 50]
        rev_text = page.get_textbox(rev_roll_pos)
        if rev_text == pn_rev:
            page.draw_rect(rev_roll_pos, width=1.5, color=(0, 0, 1))
        else:
            red_line = True
            rev_result.append(f'Page {i+1}: ({rev_text})')
            highlight = page.add_highlight_annot(rev_roll_pos)
            highlight.set_colors({"stroke":(1, 0, 0)})
            highlight.update()

        # check key in page
        if key_check != '':
            key_found = []
            for key in key_check:
                search_founds = page.search_for(key)
                if len(search_founds) > 0:
                    key_found.append(key)
                    red_line = True
                    for item in search_founds:
                        highlight = page.add_highlight_annot(item)
                        highlight.set_colors({"stroke":(1, 0, 0)})
                        highlight.update()
            if len(key_found) > 0:
                key_check_result.append(f'page {i+1}: {"/".join(key_found)}')

        if i == 0:
            des_pos = [page.rect.width - 300,page.rect.height - 130,page.rect.width - 32,page.rect.height - 70]
            rev_table_pos = []
            size_pos = page.search_for('size',clip = title_block)
            if len(size_pos) == 1:
                print('found size',size_pos)
                des_pos[0] = size_pos[0][0] - 2
#             else:
#                 print('can not find size')
#                 des_pos.append(page.rect.width - 300)

            sign_pos = page.search_for('signature',clip = title_block)
            if len(sign_pos) == 1:
                des_pos[1] = sign_pos[0][3]+25
#             else:
#                 print('can not find des_0')
#                 des_pos.append(page.rect.height - 130)

            if len(rev_pos) == 1:
                des_pos[2] = rev_pos[0][2] + 5
                des_pos[3] = rev_pos[0][1] - 5
                print('Found Rev position')
            elif len(size_pos) > 0:            
                des_pos[3] = size_pos[0][1] - 5
#                 else:
#                     des_pos.append(page.rect.height - 70)
            print('des_pos',des_pos)
            page.draw_rect(des_pos, width=1.5, color=(1, 0, 0))
            des_text = page.get_textbox(des_pos).replace('\n','').strip()
            print('des_pos',des_pos,'des_text',des_text)
            if len(des_text) > 40:
                red_line = True
                des_result = 'Des. len > 40 (' + des_text + ')'
                highlight = page.add_highlight_annot(des_pos)
                highlight.set_colors({"stroke":(1, 0, 0)})
                highlight.update()
            elif des_text == des_input or des_input == '':
                page.draw_rect(des_pos, width=1.5, color=(0, 0, 1))
                des_result = des_text
            else:
                red_line = True
                des_result = 'Des not match (' + des_text + ')'
                highlight = page.add_highlight_annot(des_pos)
                highlight.set_colors({"stroke":(1, 0, 0)})
                highlight.update()

            # extract rev table in page 1
    #         if pn_rev != 'AA':
            rev_block = [page.rect.width - 600, 35 ,page.rect.width - 30, 200]
            page.draw_rect(rev_block, width=1.5, color=(0, 1, 1))
            des_rev_table = page.search_for('description', clip = rev_block)
            print('des_rev_table',des_rev_table)
            if len(des_rev_table) == 1:
                print('Len des_rev_table',len(des_rev_table))
                rev_block[0] = des_rev_table[0][0] - 200
                rev_block[1] = des_rev_table[0][1] - 2
                rev_block[3] = des_rev_table[0][3] + 50
            page.draw_rect(rev_block, width=1.5, color=(0.5, 1, 0))
            rev_column = page.search_for('rev', clip = rev_block)
            page.draw_rect(rev_block, width=1.5, color=(1, 0.5, 0))
            if len(rev_column) == 1:
                print('Len rev_column',len(rev_column))
                rev_block[0] = rev_column[0][0] - 2
                rev_block[1] = rev_column[0][3]
                rev_block[2] = rev_column[0][2] + 2
                rev_block[3] = rev_column[0][3] +  page.rect.height/3
            else:

                eco_pos = page.search_for('eco', clip = rev_block)
                if len(eco_pos) == 1:
                    print('Len eco_pos',len(eco_pos))
                    rev_block[0] = eco_pos[0][0] + 30
                    rev_block[1] = eco_pos[0][3]
                    rev_block[2] = eco_pos[0][2] + 45
                    rev_block[3] = eco_pos[0][3] +  page.rect.height/3
                else:
                    print('can not found rev position in table')
                    rev_block[0] += 150
                    rev_block[1] += 5
                    rev_block[2] -= 330
                    rev_block[3] +=  page.rect.height/4
            rev_list = page.get_textbox(rev_block)
            if pn_rev in rev_list:
                page.draw_rect(rev_block, width=1.5, color=(0, 0, 1))
            else:
                rev_table_result = 'Rev not update in Revision table, pls check [' + rev_list +']'
                page.draw_rect(rev_block, width=1.5, color=(1, 0, 0))
                print('Rev not update in Revision table, pls check',rev_list)
    if len(pn_result) > 0:
        pn_result = 'PN not match in: ' + '; '.join(pn_result)
    else: pn_result = pn_text

    if len(rev_result) > 0:
        rev_result = 'Rev not match in: ' + '; '.join(rev_result)
    else: rev_result = rev_text
    if rev_table_result != '':
        rev_result = rev_result + ' (' + rev_table_result +')'
    if len(key_check_result) > 0:
        key_check_result = 'Found key in: ' + '; '.join(key_check_result)
    else: key_check_result = ''        

    print('des_result:',des_result)
    print('pn_result:',pn_result)
    print('rev_result:',rev_result)
    print('key_check_result:',key_check_result)
    if red_line:
        viz_name = os.path.join(file_path, file[:-4] + "________Redline.pdf" )
    else:
        viz_name = os.path.join(file_path, file[:-4] + "_SelfQC.pdf" )
    doc.save(viz_name)

    return pn_result,rev_result,des_result,key_check_result

## Check para on ECO

In [216]:
# driver.switch_to.window(ECO_win)
# driver.switch_to.window(driver.window_handles[-1])
# ECO_win = driver.current_window_handle

In [228]:
def get_info_on_eco(pn_input, des_input):

    pn_enovia_result = ''
    rev_enovia_result = ''
    des_enovia_result = ''
    uploaded_result = ''
    state_result = ''
    rohs_result = ''
    attached_found = False
    attached_result = ''
    
    # des_enovia_result = ''
    # uploaded_result = ''
    # rohs_result = ''
    # state_result = ''
    # pn_enovia_result = ''
    # rev_enovia_result = ''
    # rohs_result = ''
    # attached_found = False
    # attached_result = ''
    pn, pn_key, pn_dash, pn_rev = ext_pn(pn_input)
    print('PN Detail: ',pn, pn_key, pn_dash, pn_rev)
    print('has eco num, go to ECO Win')
    # try: 
    driver.switch_to.window(ECO_win)
    print('Drive title: ',driver.title)
    # except: goto_ECO_btn()
    # driver.switch_to.window(ECO_win)
    # # driver.maximize_window()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    # driver.switch_to.frame('content')
    # affected_items_bar = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
    #                                         '//*[@id="tvcTabs0"]/div[1]/ul/li[2]/div/a[1]')))
    # affected_items_bar.click()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    for k in range(len(affected_items)):
        uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                        + str(k+1) + ']/td[28]')
        found_uom = re.findall(r'\w',uom.text)
    #                 print(found_uom)
        if len(found_uom) < 1:
            continue
        else:
            new_parts = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                        + str(k+1) + ']/td[18]')
    #                     print(new_parts.text)
            if pn_key in new_parts.text:
                print('found part',pn_key)
                if pn == new_parts.text:
                    print('dash roll match',pn)
                    pn_enovia_result = new_parts.text
                else:
                    pn_enovia_result = 'PN not match (' + new_parts.text + ')'
                rev_new_enovia = driver.find_element(By.XPATH,\
                            '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                    + str(k+1) + ']/td[19]')
                if pn_rev == rev_new_enovia.text:
                    print('rev match',pn_rev)
                    rev_enovia_result = rev_new_enovia.text

                elif pn_rev == "":
                    rev_enovia_result = 'Enovia new Rev: ' + rev_new_enovia.text
                else:
                    print('rev not match')
                    rev_enovia_result = '(' + rev_new_enovia.text + ') Not match'
                attached = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                            + str(k+1) + ']/td[8]')
                attached_num = re.findall(r'\d',attached.text)
                if len(attached_num) > 0:
                    attached_found = True
                    attached_result = str(attached_num[0]) + ' Attached'
                    print('attached_result:',attached_result)
                else:
                    attached_found = False
                    attached_result = 'No older Attached'
                    print('No Attached')
    #                     print(attached_result)
                if attached_found:
                    uploaded_field = driver.find_element(By.XPATH, \
                    '/html/body/form/div[3]/div[2]/table/tbody/tr[' + str(k+1) + ']/td[23]')
                    uploaded_num = re.findall(r'\d',uploaded_field.text)
                    if int(uploaded_num[0]) > 0:
                        if attached_num[0] == uploaded_num[0]:
                            uploaded_result = uploaded_num[0] + ' files uploaded'
                        else:
                            uploaded_result = uploaded_num[0] + ' files uploaded not match with previous'
                    else:
                        uploaded_result = 'Not yet'
                else:
                    uploaded_result = '-'
    
                des_new_enovia = driver.find_element(By.XPATH,\
                    '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                            + str(k+1) + ']/td[20]/span')
                if des_input == '':
                    des_enovia_result = 'Des. on Enovia: ' + '(' + des_new_enovia.text + ')'
                elif des_input == des_new_enovia.text:
                    des_enovia_result = des_new_enovia.text
                else:
                    des_enovia_result = 'Des not match ' + '(' + des_new_enovia.text + ')'
                rosh = driver.find_element(By.XPATH,\
                    '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                            + str(k+1) + ']/td[30]')
                rohs_result = rosh.text
    
                state = driver.find_element(By.XPATH,\
                    '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                            + str(k+1) + ']/td[22]')
                state_result = state.text
    
                break
    #         print(pn_enovia_result,rev_enovia_result,des_enovia_result,attached_result,rohs_result,state_result)
    
    
    if pn_enovia_result == '':
        # print('part_detail: ', part_detail)
        found_add = False
        for part in part_detail:
            if pn_key in part[0] and part[1] == 'add':
                found_add = True
                pn_enovia_result = 'New added part in Affected BOMs'  
                break
            elif pn_key in part[0] and part[1] == 'replace':
                found_add = True
                pn_enovia_result = 'Replaced part in Affected BOMs'  
                break
        if found_add:
            print(f'found Pn {pn} is replace or add')
            # check_add_pn(i,ws,pn)
            # thread = threading.Thread(target = check_add_pn, \
            #     args = (i,ws,pn, 10000))
            # thread.start()
        # for line in lines:
        #     if pn_key in line:
        #         if 'add' in line.lower():
        else:
            pn_enovia_result = 'Not found in Affected items'        

    return pn_enovia_result, \
            rev_enovia_result,\
            des_enovia_result, \
            uploaded_result,\
            state_result, \
            rohs_result, \
            attached_found, \
            attached_result
# pn_input = ' 0780606-000'
# des_input = 'ASSY, OUTER FRAME, SP7'
# pn_enovia_result,\
# rev_enovia_result,\
# des_enovia_result,\
# uploaded_result,\
# state_result,\
# rohs_result,\
# attached_found,\
# attached_result\
# = get_info_on_eco(pn_input, des_input)
# print(f' des_enovia_result: {des_enovia_result}\n uploaded_result: {uploaded_result}\n rohs_result: {rohs_result}\n\
#  state_result: {state_result}\n pn_enovia_result: {pn_enovia_result}\n \
# rev_enovia_result: {rev_enovia_result},\n rohs_result: {rohs_result}\n attached_found: {attached_found}\n\
#  attached_result: {attached_result}')

PN Detail:  0780606-000 0780606 000 
has eco num, go to ECO Win
Drive title:  ENOVIA
found part 0780606
dash roll match 0780606-000
No Attached
 des_enovia_result: Des not match (ASSY, OUTER FRAME, SP7+)
 uploaded_result: -
 rohs_result: Unassigned
 state_result: Preliminary
 pn_enovia_result: 0780606-000
 rev_enovia_result: Enovia new Rev: AK,
 rohs_result: Unassigned
 attached_found: False
 attached_result: No older Attached


## Main check para

In [218]:
def main_para_check():
    # wb = xw.Book(file_parameter_name)
    # ws = wb.sheets.active
    lr = ws.range('B' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
    if len(entry_ECO_num.get().strip()) > 4:
        eco_num = entry_ECO_num.get().strip()
    else:
        eco_num = None
    print('ECO number: ',eco_num,eco_num is not None)
    # try: 
    #     print('ECO number: ',ECO_number)
    # except: 
        
    #     ECO_number = None
    #     print('ECO set to None: ',ECO_number)
    # eco_num = entry_ECO_num.get().strip()
    # if len(eco_num) < 5:
    #     eco_num = None
    if eco_num is not None: 
        print('has eco num, go to ECO Win')
        try: driver.switch_to.window(ECO_win)
        except: goto_ECO_btn()
        driver.switch_to.window(ECO_win)
        # driver.maximize_window()
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        # driver.switch_to.frame('content')
        affected_items_bar = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="tvcTabs0"]/div[1]/ul/li[2]/div/a[1]')))
        affected_items_bar.click()
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
        affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    
    for i in range(2,lr+1):
        print('collecting for:',ws.range(i,2).value)
        if  ws.range(i,2).value == None: continue #ws.cell(row=i, column=2).value == None:
    #         if len(ws.range(i,2).value) < 16: #len(ws.cell(row=i, column=2).value) < 16:
        
        pn_input = ws.range(i,2).value.strip()
        # pn_rev = pn_input[12:14]
        pn, pn_key, pn_dash, pn_rev = ext_pn(pn_input)
    #         else: continue
        if ws.range(i,3).value != None:
            des_input = ws.range(i,3).value.strip()
        else:
            des_input = ''
    
        des_enovia_result = ''
        uploaded_result = ''
        rohs_result = ''
        state_result = ''
        pn_enovia_result = ''
        rev_enovia_result = ''
        rohs_result = ''
        attached_found = False
        attached_result = ''
    
        if eco_num is not None:
    #             print('eco not none')
            pn_enovia_result,\
            rev_enovia_result,\
            des_enovia_result,\
            uploaded_result,\
            state_result,\
            rohs_result,\
            attached_found,\
            attached_result\
            = get_info_on_eco(pn_input, des_input)
    # #             affected_items = driver.find_elements(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr')
    #         for k in range(len(affected_items)):
    #             uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                             + str(k+1) + ']/td[28]')
    #             found_uom = re.findall(r'\w',uom.text)
    # #                 print(found_uom)
    #             if len(found_uom) < 1:
    #                 continue
    #             else:
    #                 new_parts = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                             + str(k+1) + ']/td[18]')
    # #                     print(new_parts.text)
    #                 if pn_key in new_parts.text:
    #                     print('found part',pn_key)
    #                     if pn == new_parts.text:
    #                         print('dash roll match',pn)
    #                         pn_enovia_result = new_parts.text
    #                     else:
    #                         pn_enovia_result = 'PN not match (' + new_parts.text + ')'
    #                     rev_new_enovia = driver.find_element(By.XPATH,\
    #                                 '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                                         + str(k+1) + ']/td[19]')
    #                     if pn_rev == rev_new_enovia.text:
    #                         print('rev match',pn_rev)
    #                         rev_enovia_result = rev_new_enovia.text
    #                     else:
    #                         print('rev not match')
    #                         rev_enovia_result = '(' + rev_new_enovia.text + ') Not match'
    #                     attached = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                                 + str(k+1) + ']/td[8]')
    #                     attached_num = re.findall(r'\d',attached.text)
    #                     if len(attached_num) > 0:
    #                         attached_found = True
    #                         attached_result = str(attached_num[0]) + ' Attached'
    #                         print('attached_result:',attached_result)
    #                     else:
    #                         attached_found = False
    #                         attached_result = 'No older Attached'
    #                         print('No Attached')
    # #                     print(attached_result)
    #                     if attached_found:
    #                         uploaded_field = driver.find_element(By.XPATH, \
    #                         '/html/body/form/div[3]/div[2]/table/tbody/tr[' + str(k+1) + ']/td[23]')
    #                         uploaded_num = re.findall(r'\d',uploaded_field.text)
    #                         if int(uploaded_num[0]) > 0:
    #                             if attached_num[0] == uploaded_num[0]:
    #                                 uploaded_result = uploaded_num[0] + ' files uploaded'
    #                             else:
    #                                 uploaded_result = uploaded_num[0] + ' files uploaded not match with previous'
    #                         else:
    #                             uploaded_result = 'Not yet'
    #                     else:
    #                         uploaded_result = '-'
    
    #                     des_new_enovia = driver.find_element(By.XPATH,\
    #                         '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                                 + str(k+1) + ']/td[20]/span')
    #                     if des_input == '':
    #                         des_enovia_result = 'Des. on Enovia: ' + '(' + des_new_enovia.text + ')'
    #                     elif des_input == des_new_enovia.text:
    #                         des_enovia_result = des_new_enovia.text
    #                     else:
    #                         des_enovia_result = 'Des not match ' + '(' + des_new_enovia.text + ')'
    #                     rosh = driver.find_element(By.XPATH,\
    #                         '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                                 + str(k+1) + ']/td[30]')
    #                     rohs_result = rosh.text
    
    #                     state = driver.find_element(By.XPATH,\
    #                         '/html/body/form/div[3]/div[2]/table/tbody/tr['\
    #                                 + str(k+1) + ']/td[22]')
    #                     state_result = state.text
    
    #                     break
    # #         print(pn_enovia_result,rev_enovia_result,des_enovia_result,attached_result,rohs_result,state_result)
    
    
    #         if pn_enovia_result == '':
    #             # print('part_detail: ', part_detail)
    #             found_add = False
    #             for part in part_detail:
    #                 if pn_key in part[0] and part[1] == 'add':
    #                     found_add = True
    #                     pn_enovia_result = 'New added part in Affected BOMs'  
    #                     break
    #                 elif pn_key in part[0] and part[1] == 'replace':
    #                     found_add = True
    #                     pn_enovia_result = 'Replaced part in Affected BOMs'  
    #                     break
    #             if found_add:
    #                 print(f'found Pn {pn} is replace or add')
    #                 # check_add_pn(i,ws,pn)
    #                 thread = threading.Thread(target = check_add_pn, \
    #                     args = (i,ws,pn, 10000))
    #                 thread.start()
    #             # for line in lines:
    #             #     if pn_key in line:
    #             #         if 'add' in line.lower():
    #             else:
    #                 pn_enovia_result = 'Not found in Affected items'        
    
    
        pn_result = ''
        rev_result = ''
        des_result = ''
        key_check_result = ''
        file_name_result = ''
        drw_found = False  
    #     if attached_found == True\
    #     or eco_num is None:
    
        # print(ppn_key,'file:',file)
        for file in os.listdir(file_path_para):
            if (file.lower().endswith(".pdf"))\
            and (pn_key in file)\
            and (len(file) < 23): #and (pn_rev in file):
    #                     print('input file',file)
    
                drw_found = True
                print('found Drawing in folder')
                print('pn_input[8:11]',pn_dash)
                if (pn_dash not in file[7:12])\
                or (pn_rev not in file[9:15]):
                    file_name_result = 'File name not match ' + '(' + file + ')'
                pn_drw = os.path.join(file_path_para, file)
                doc: Document = fitz.open(pn_drw)
                page: Page = doc[0]
                clip_drw_pos =\
                [page.rect.width - 420,page.rect.height - 180,page.rect.width - 30,page.rect.height - 40]
                clip_oms_pos = [0,0,200,100]
                initial_text = page.get_text()
    #                 highlight = page.add_highlight_annot(clip_drw_pos)
    #                 highlight.set_colors({"stroke":(0, 0.6, 0.8)})
    #                 highlight.update()
    #                 viz_name = os.path.join(file_path, file[:-4] + "check.pdf" )
    #                 doc.save(viz_name)
                print('initial_text len',len(initial_text),initial_text)
                if page.rect.width < page.rect.height:
                    print('Drawing file is Portable, can not check')
                    pn_result = 'Drawing file is Portable, can not check'
                elif len(initial_text) < 1:
                    print('can not get text in the file')
                    pn_result = 'can not get text in the file'
                else:
                    drw_num = page.search_for('drawing number',clip = clip_drw_pos)
                    doc_num = page.search_for('document number',clip = clip_oms_pos)
    #                     print(drw_num,'page width:',page.rect.width,'page height:',page.rect.height)
                    if len(doc_num) > 0:
                    #     print('can not find part number in page')
                    # elif (pns_found[0][0] < page.rect.width / 2): # and (pns_found[0][1] < page.rect.height / 2):
                        print('file is OMS')
                        pn_result,rev_result,des_result,key_check_result\
                            = oms_extract(file_path_para, file, pn_input,des_input, key_check)
                    elif len(drw_num) > 0: # (drw_num[0][0] > page.rect.width / 2):
                        print('file is Drawing')
                        pn_result,rev_result,des_result,key_check_result\
                            = drw_extract(file_path_para, file, pn_input,des_input, key_check)
                    else:
                        pn_result = 'Document found but can not read'
                # break after got Doc.
                break
    
        if drw_found == False and attached_found:
            pn_result = 'Not found Drawing'
    #     if rev_not_match:
    #         rev_result = rev_text + ' Not match Rev'
    #     print('input Des.:',des_input)
    #     if 'oms' in des_input.lower():
    
        if len(re.findall(r'\bkit\b',des_input.lower())) > 0:
            result = 'Enovia: KIT\n-' + attached_result + '\n-' + pn_enovia_result + ' '\
            + rev_enovia_result + '\n-' \
            + des_enovia_result
    #         elif len(re.findall(r'\boms\b',des_input.lower())) > 0:
    #             result = 'Enovia: OMS\n-' + attached_result + '\n-' + pn_enovia_result + ' '\
    #             + rev_enovia_result + '\n-' \
    #             + des_enovia_result
        elif len(re.findall(r'\bproc\b',des_input.lower())) > 0\
        or len(re.findall(r'\bprocedure\b',des_input.lower())) > 0:
            result = 'Enovia: PROC\n-' + attached_result + '\n-' + pn_enovia_result + ' '\
            + rev_enovia_result + '\n-' \
            + des_enovia_result
        elif len(re.findall(r'\bpkg\b',des_input.lower())) > 0:
            result = 'Enovia: PKG\n-' + attached_result + '\n-' + pn_enovia_result + ' '\
            + rev_enovia_result + '\n-' \
            + des_enovia_result
    #     elif attached_found == False\
    #     and eco_num is not None:
    #         result = 'Enovia:\n-' + attached_result + '\n-' + pn_enovia_result + ' '\
    #         + rev_enovia_result + '\n-' \
    #         + des_enovia_result
        elif eco_num is None:
            result = \
            'Document__________:\n-'\
            + file_name_result + '\n-'\
            + pn_result + ' '\
            + rev_result + '\n-' + des_result + '\n-' + key_check_result
        else:
            result = 'Enovia:\n-' + attached_result + '\n-' + pn_enovia_result + ' '\
            + rev_enovia_result + '\n-' \
            + des_enovia_result + '\n' \
            + 'Document__________:\n-'\
            + file_name_result + '\n-'\
            + pn_result + ' '\
            + rev_result + '\n-' + des_result + '\n-' + key_check_result
    
        ws.range(i,6).value = result
        ws.range(i,8).value = uploaded_result
        ws.range(i,10).value = rohs_result
        ws.range(i,12).value = state_result
    if eco_num is not None:
        plant_status_check()

In [219]:
def check_add_pn(i,ws,pn, time):
    print('threading ______________',i,ws,pn, time)
    # def create_popup(title, mes, time):
    # popup = tk.Toplevel(window)
    # popup.attributes('-topmost',True)
    # popup.title(title)
    # popup.geometry("300x150")
    # popup_label = tk.Label(popup, text = i + ' ' + pn)
    # popup_label.pack(pady=20)
    # popup.after(time, lambda: close_popup(popup))

In [220]:
# check_add_pn(1,'ad','000',10000)

## Plant status

In [221]:
# if (file_type  == '.xlsx') and 'parameter' in file_name.lower():
def plant_status_check():
    sleep(random.randint(1, 2))
    driver.switch_to.window(ECO_win)
    # eco_win = driver.current_window_handle
    driver.maximize_window()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    sap_bar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'SAP Initial Data')))
    sap_bar.click()
#         sleep(random.randint(5, 6))
    driver.switch_to.default_content()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBottomFrame'))
    pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
             '/html/body/div/div[4]/table/tbody/tr/td/table/tbody/tr/td[4]/img[2]')))

    pagination.click()

    driver.switch_to.default_content()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    sap_list = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
#         lr = len(ws['B'])
    lr = ws.range('B' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
    pn = ''

    for i in range(2,lr+1):
        if ws.range(i,2).value == None:
            print('cell  None:',i)
            continue
        if len(ws.range(i,2).value) < 16:
            pn, pn_key, pn_dash, pn_rev = ext_pn(ws.range(i,2).value.strip())
            # pn = ws.range(i,2).value.strip()[:11]
        plant_result = ''
        found_plant = False
        for j in range(len(sap_list)):
            if pn in sap_list[j].text:
                found_plant = True
#                 print(i,'pn:',pn)
                plant_status = driver.find_element(By.XPATH,\
                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                    + str(j+1) + ']/td[11]')
                plant_result = plant_status.text
#                 print(pn,'plant:',plant_status.text)
                break
        if found_plant == False:
            for j in range(len(sap_list)):
                if pn_key in sap_list[j].text:
                    plant_result = 'Not match Dash rolled'
                    break
        ws.range(i,9).value = plant_result
    print('Parameter check done')
    tk.messagebox.showinfo(title='Message', message='Parameter checked, Done')

## Check Parameter Button

In [222]:
def check_para_btn():
    ext_script_to_excel()
    main_para_check()

In [223]:

# # def check_replaced_pn(pn_key):
# pn_key = '0608053*'
# get_main_page()
# # driver.switch_to.window(main_page)
# search_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
#                                                     '//*[@id="quickSearch"]')))
# search_field.send_keys(pn_key)
# #     sleep(random.randint(0, 1))
# search_field.send_keys(Keys.RETURN)
# driver.switch_to.window(driver.window_handles[-1])
# result_page = driver.current_window_handle
# wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
# #     driver.switch_to.frame('tableContentFrame')
# driver.switch_to.frame('tableBodyRight')
# search_results = driver.find_elements(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr')
# print(search_results)
# #     if len(ecos) == 1:
# #     #     print('one ECO found')
# #         eco_select = driver.find_element(By.XPATH, '//*[@id="tbr"]/tbody/tr/td/a')
# #     else:
# #         print('no or more than one ECO found')
# #         eco_select_number = simpledialog.askstring(title="Script to Enovia",
# #                                       prompt="Please select ECO, input 1, 2 or 3:")
# #         eco_select = driver.find_element(By.XPATH,\
# #                     '//*[@id="tbr"]/tbody/tr[' + str(eco_select_number) + ']/td/a')
# #     eco_select.click()
# #     sleep(5)
# #     driver.switch_to.window(driver.window_handles[-1])
# #     # eco_win_return = driver.current_window_handle
# # #     driver.switch_to.window(eco_win)
# # # #     driver.maximize_window()
# #     wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
# #     affected_bar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
# #                                     'All Affected Items')))
# #     affected_bar.click()
# #     sleep(1)
# #     driver.switch_to.window(driver.window_handles[-1])
# #     eco_win_return = driver.current_window_handle


# # check_replaced_pn(pn_key)

In [224]:
# pn = '0608053-005'
# for i in range(len(search_results)-1,0, -1):
#     pn_in_result = driver.find_element(By.XPATH, \
#                 '/html/body/form/div[2]/div[2]/table/tbody/tr[' + str(i) + ']/td[4]')
#     fru_in_result = driver.find_element(By.XPATH, \
#                 '/html/body/form/div[2]/div[2]/table/tbody/tr[' + str(i) + ']/td[7]')
#     plant_sent_result = driver.find_element(By.XPATH, \
#                 '/html/body/form/div[2]/div[2]/table/tbody/tr[' + str(i) + ']/td[15]')
#     print(pn_in_result.text,fru_in_result.text,plant_sent_result.text)
#     if pn_in_result.text == pn and (fru_in_result.text != '' or plant_sent_result.text != ''):
#         print('found: ',pn_in_result.text,fru_in_result.text,plant_sent_result.text)

# Windown Display

In [225]:
window = tk.Tk()
window.title('ECO Supporting')
window.iconbitmap('Capture.ico')
### window size
win_width = 600
win_height = 400
window.geometry(str(win_width) + 'x' + str(win_height))
window.resizable(0, 0)
    
### y vertical position
y_1 = 0.1
y_2 = 0.4
y_3 = 0.7

### x horizontal position
x_left_side = 0.1
x_right_side = 0.9
x_mid = 0.59
### rel of x of entries
relx_entry = 0.32

### button size
btn_width = 100
# btn_height = 20

### label font
font_label = 'Calibri 11 bold'

### font Header 
font_header = 'Calibri 11 bold'
# font_btn = font.Font(family='Helvetica', size=16, weight='bold')

### Frame script writting ______________________________________________
bg_frame_script = "#CDD7D6"
frame_script = tk.Frame(window, background= bg_frame_script)
frame_script.pack(expand = True, fill=BOTH)



### label ECO request form
label_eco_req_form = ttk.Label(frame_script, text = 'ECO Request Form',\
            background = bg_frame_script,\
            font =font_label)

### label label_plant_pro_key
label_plant_pro_key=ttk.Label(frame_script, text = 'Project name-Code',\
            background = bg_frame_script,\
            font =font_label)
### entry request form
var_req_form_str = tk.StringVar()
var_req_form_str.trace("w", lambda name, index, mode,\
            var_req_form_str=var_req_form_str: req_form_value_track(var_req_form_str))
entry_req_form = ttk.Entry(master = frame_script, textvariable=var_req_form_str)

### entry plant project key
entry_plant_pro_key = ttk.Entry(master = frame_script)


### button brower
btn_brw_req_form = ttk.Button(master = frame_script, text = 'Browser', command = get_form_file)

### check download variable
var_check_down = tk.IntVar(value = 1)

### check box download WhereUsedBOMs
check_down = ttk.Checkbutton(frame_script, text = 'Download WUBs',variable = var_check_down)

### button write script
btn_write_script = ttk.Button(master = frame_script, text = 'Write Script', command = script_btn,\
                             state = DISABLED)

### check box plant 05 --> rev rolled only
var_check_plant = tk.IntVar(value = 1)
# var_check_plant = False
check_plant = ttk.Checkbutton(frame_script, text = 'Check plant 05',\
    variable = var_check_plant)

### place components
label_eco_req_form.place( relx = x_left_side, rely = y_1)
entry_req_form.place( relx = relx_entry, rely = y_1, width = 2*btn_width)
label_plant_pro_key.place( relx = x_left_side, rely = y_3)
entry_plant_pro_key.place( relx = relx_entry, rely = y_3, width = 2*btn_width)

btn_brw_req_form.place( relx = x_right_side, rely = y_1, width = btn_width , anchor = 'ne')
check_down.place(relx = x_left_side, rely = y_2)
check_plant.place(relx = 0.4, rely = y_2)
btn_write_script.place( relx = x_right_side, rely = y_2, width = btn_width, height=60 , anchor = 'ne')



### frame work on Enovia ______________________________________________
bg_frame_ECO = "#829D99"
frame_ECO = tk.Frame(window, background= bg_frame_ECO) #,height=win_height/3
frame_ECO.pack(expand = True, fill = BOTH)

### y vertical position
y_3 = 0.1
y_4 = 0.4

### line 1
### label ECO request form
label_script_file = ttk.Label(frame_ECO, text = 'Script file',
            font =font_label,
            background=bg_frame_ECO)


### entry script file

var_script_file_str = tk.StringVar()
var_script_file_str.trace("w", lambda name, index, mode,\
            var_script_file_str=var_script_file_str: script_file_path_track(var_script_file_str))

entry_script_file = ttk.Entry(master = frame_ECO, textvariable=var_script_file_str)
### button browser script file
btn_brw_script = ttk.Button(frame_ECO, text = 'Browser', command = get_script_file)
# btn_brw_script['font'] = lable_font

### line 2
### label ECO numeber
label_ECO_num = ttk.Label(frame_ECO, text = 'ECO Number',
        font =font_label,
         background=bg_frame_ECO)
### entry script file
var_ECO_number = tk.StringVar()
var_ECO_number.trace("w", lambda name, index, mode,\
            var_ECO_number=var_ECO_number: def_ECO_number_track(var_ECO_number))
entry_ECO_num = ttk.Entry(master = frame_ECO, textvariable=var_ECO_number)
### button browser script file
btn_goto_ECO = ttk.Button(frame_ECO, text = 'Go to ECO', command = goto_ECO_btn,\
                         state = DISABLED)

### place components
label_script_file.place( relx = x_left_side, rely = y_3)
entry_script_file.place( relx = relx_entry, rely = y_3, width = 2*btn_width)
btn_brw_script.place( relx = x_right_side, rely = y_3, width = btn_width , anchor = 'ne')
label_ECO_num.place( relx = x_left_side, rely = y_4)
entry_ECO_num.place( relx = relx_entry, rely = y_4, width = 2*btn_width)
btn_goto_ECO.place( relx = x_right_side, rely = y_4, width = btn_width , anchor = 'ne')


### frame option on Enovia ______________________________________________
bg_frame_ECO_opt = "#759B99"
frame_ECO_opt = tk.Frame(window, background= bg_frame_ECO_opt,
    highlightbackground = 'blue') #,height=win_height/3

label_execute = ttk.Label(frame_ECO_opt, text = 'Execution option',
            font =font_label,
            background=bg_frame_ECO_opt)

### not working
frame_ECO_opt.config(highlightbackground = "red", highlightcolor= "red")
frame_ECO_opt.pack(expand = True, fill = BOTH)
label_execute.place( relx = x_left_side, rely = 0.05)
### y vertical position
y_5 = 0.3
y_6 = 0.6

### column 1
### Read script btn
btn_read_script = ttk.Button(frame_ECO_opt, text = 'Read Script',\
                command = lambda: shown_BOMs_btn('List of BOMs'),\
                    state = DISABLED)

### add affected items
btn_add_items = ttk.Button(frame_ECO_opt, text = 'Add af. Items',\
                command = add_affected_items_btn,\
                    state = DISABLED)

### column 2
### Update BOM btn
btn_update_BOM = ttk.Button(frame_ECO_opt, text = 'Update BOM',\
                           state = DISABLED, command = update_BOM_btn)

### update SAP ID
btn_update_sap = ttk.Button(frame_ECO_opt, text = 'Update SAP',\
                            state = DISABLED)


### column 3
btn_check_parameter = ttk.Button(frame_ECO_opt, text = 'Check Parameter',\
                                command =check_para_btn)
btn_down_report = ttk.Button(frame_ECO_opt, text = 'Down Reports',\
                             state = DISABLED)

### place components
btn_read_script.place( relx = x_left_side, rely = y_5,width = btn_width)
btn_add_items.place( relx = x_left_side, rely = y_6, width = btn_width)
btn_update_BOM.place( relx = x_mid, rely = y_5, width = btn_width , anchor = 'ne')
btn_update_sap.place(relx = x_mid, rely = y_6,width = btn_width, anchor = 'ne')
btn_down_report.place( relx = x_right_side, rely = y_5, width = btn_width , anchor = 'ne')
btn_check_parameter.place( relx = x_right_side, rely = y_6, width = btn_width , anchor = 'ne')



### frame copyright ______________________________________________
bg_frame_copyright = "#495B59"
frame_copyright = tk.Frame(window, background= bg_frame_copyright, height = 30) #,height=win_height/3
frame_copyright.pack(fill=BOTH)

### line 1
### button browser script file
label_copyright = ttk.Label(frame_copyright, text = 'Copyright \xa9 KLA +',
                    background=bg_frame_copyright)

### place components
label_copyright.pack(side = LEFT)

window.mainloop()

key_check []
extract data from script file: 24-01-16_script_auto.txt
dash pn has in pn_list:  0803519-006
release pn has in pn_list:  0803519-000
children release list pn has in pn_list:  1095838-000
children release list pn has in pn_list:  1069521-000
children release list pn has in pn_list:  1054887-000
can not extract for line: 4. Below parts extended to 3008 plant:

can not extract for line: 4. Below parts extended to 3008 plant:

break at:  38
can not extract for line: 4. Below parts extended to 3008 plant:

children release list pn has in pn_list:  1074546-000
extract data from script file: 24-01-23_script_auto.txt
dash pn has in pn_list:  0580970-017
dash pn has in pn_list:  0855644-009
dash pn has in pn_list:  0685109-020
dash pn has in pn_list:  0580970-017
dash pn has in pn_list:  0855644-009
dash pn has in pn_list:  0991198-004
dash pn has in pn_list:  0685109-020
rev pn has in pn_list:  0844534-000
rev pn has in pn_list:  0780606-000
dash pn has in pn_list:  0991203-004
ex

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\dontien\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\dontien\AppData\Local\Temp\ipykernel_32\2735812979.py", line 3, in check_para_btn
    main_para_check()
  File "C:\Users\dontien\AppData\Local\Temp\ipykernel_32\2498715592.py", line 69, in main_para_check
    = get_info_on_eco(pn_input, des_input)
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dontien\AppData\Local\Temp\ipykernel_32\4288315885.py", line 39, in get_info_on_eco
    uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dontien\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 740, in find_element
    return self.execute(Comm

key_check []
extract data from script file: 24-01-16_script_auto.txt
dash pn has in pn_list:  0803519-006
release pn has in pn_list:  0803519-000
children release list pn has in pn_list:  1095838-000
children release list pn has in pn_list:  1069521-000
children release list pn has in pn_list:  1054887-000
can not extract for line: 4. Below parts extended to 3008 plant:

can not extract for line: 4. Below parts extended to 3008 plant:

break at:  38
can not extract for line: 4. Below parts extended to 3008 plant:

children release list pn has in pn_list:  1074546-000
extract data from script file: 24-01-23_script_auto.txt
dash pn has in pn_list:  0580970-017
dash pn has in pn_list:  0855644-009
dash pn has in pn_list:  0685109-020
dash pn has in pn_list:  0580970-017
dash pn has in pn_list:  0855644-009
dash pn has in pn_list:  0991198-004
dash pn has in pn_list:  0685109-020
rev pn has in pn_list:  0844534-000
rev pn has in pn_list:  0780606-000
dash pn has in pn_list:  0991203-004
ex

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\dontien\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\dontien\AppData\Local\Temp\ipykernel_32\2735812979.py", line 3, in check_para_btn
    main_para_check()
  File "C:\Users\dontien\AppData\Local\Temp\ipykernel_32\2498715592.py", line 69, in main_para_check
    = get_info_on_eco(pn_input, des_input)
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dontien\AppData\Local\Temp\ipykernel_32\4288315885.py", line 39, in get_info_on_eco
    uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dontien\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 740, in find_element
    return self.execute(Comm

key_check []
extract data from script file: 24-01-16_script_auto.txt
dash pn has in pn_list:  0803519-006
release pn has in pn_list:  0803519-000
children release list pn has in pn_list:  1095838-000
children release list pn has in pn_list:  1069521-000
children release list pn has in pn_list:  1054887-000
can not extract for line: 4. Below parts extended to 3008 plant:

can not extract for line: 4. Below parts extended to 3008 plant:

break at:  38
can not extract for line: 4. Below parts extended to 3008 plant:

children release list pn has in pn_list:  1074546-000
extract data from script file: 24-01-23_script_auto.txt
dash pn has in pn_list:  0580970-017
dash pn has in pn_list:  0855644-009
dash pn has in pn_list:  0685109-020
dash pn has in pn_list:  0580970-017
dash pn has in pn_list:  0855644-009
dash pn has in pn_list:  0991198-004
dash pn has in pn_list:  0685109-020
rev pn has in pn_list:  0844534-000
rev pn has in pn_list:  0780606-000
dash pn has in pn_list:  0991203-004
ex